{/* cspell:ignore nonrepudiation, Diffie, cryptosystems, ECDH, ciphertext, Rivest, Shamir, modexp, coprime, Nino, Diaz, Ciphertext, ciphertexts, Adleman, primality, primalitytest, totient, congrel, multinv, exteuc, intrep, semsec, Coprime, varphi, modinv, kems, Fernet, OAEP, HKDF, urandom, hkdf, Prehashed, primefactor, superpoly, subexponential, subexp, gnfs, Oded, Regev, Ragavan, Vaikuntanathan, Seyoon, Ragavanm, Vinod, Regev’s, polylogarithmic, polyalg, GNFS, Ekerå, counterparty’s, ECDLP, cofactor, Larasati, counterparty's */}

# Asymmetric key cryptography

In this lesson we will look at asymmetric key cryptography which forms the basis of many secure network interactions today.

By the end of the lesson we will have covered:
 - What asymmetric key cryptography is
 - Usage of asymmetric key cryptography, including key exchange and digital signatures
 - Security of asymmetric key cryptography in general
 - Further details on RSA, DSA and Elliptic Curves algorithms and security
 - Some Python code examples showing how the algorithms work in practice
 - Threats to these algorithms from both classical and quantum computers

## Introduction to asymmetric key cryptography

As we learned in the last lesson, symmetric key cryptography is very fast and efficient for protecting information, but it has a few limitations:

1. As the number of parties wishing to exchange secure information increases, the numbers of keys required grows combinatorially. It provides no mechanism to securely distribute these keys between senders and receivers.
2. There is no provision for [*non-repudiation*][nonrepudiation]. Any party is able to decrypt, or encrypt, messages with no way to guarantee a message was received or where it originated

The solution to both these problems is provided by *asymmetric key cryptography* (AKC), also known as [*public key cryptography*][59] (PKC), which therefore forms a cornerstone of modern digital security.

Asymmetric key cryptography (AKC) involves the use of a pair of keys – one public, one private. The public and private keys are cryptographically linked and typically generated at the same time as a *key pair* using a specialized mathematical algorithm. The public key, as suggested by its name, is then meant to be freely distributed, while the private key is kept secret by the party generating the key pair. Security of communications employing asymmetric key pairs is assured as long as the private key remains confidential.

![Figure 1. Asymmetric Key Encryption](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/akc.avif)

*Figure 1. Asymmetric Key Encryption*

AKC offers several useful functions, such as:

1. **Encryption and decryption** to enable **confidentiality** of communications.
2. **Digital signatures** to ensure **authenticity**, **integrity**, and **non-repudiation**.
3. **Secure key exchange** to facilitate subsequent use of symmetric cryptosystems.

In modern applications, AKC is primarily used for digital signatures and secure key exchange. In this lesson, we introduce these two key functions, and then we discuss several variations of cryptographic protocols for these functions.

[nonrepudiation]: #definition-tooltip "Non-repudiation refers to the ability to prove that a specific party has sent a message. In symmetric key cryptography, since the same key is used for both encryption and decryption, it is not possible to determine which party has created a particular ciphertext. In contrast, asymmetric key cryptography provides non-repudiation through the use of digital signatures."
[59]: #definition-tooltip "Reference: Public-key cryptography,Wikipedia. Jul. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Public-key_cryptography&oldid=1163952916)"

## Key exchange with asymmetric key cryptography

One of the fundamental problems in cryptography is securely [exchanging keys][60]. For example, if two parties want to use symmetric encryption, both parties need the same key to encrypt and decrypt messages. But how do they securely exchange the key? Asymmetric key cryptography addresses this through interactive and non-interactive key exchange mechanisms.

[60]: #definition-tooltip "Reference: Key exchange,Wikipedia. Jan. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Key_exchange&oldid=1134576107)"

### Interactive key exchange

An interactive *key exchange protocol* refers to a method where two parties collaborate to create a *shared secret* key over an insecure communication channel. This shared secret key can then be used for symmetric encryption and decryption tasks.

The most well known among such protocols is the [Diffie-Hellman algorithm][62] (DH), which was devised specifically to facilitate key exchange. In this protocol, each party generates a pair of keys (public and private) and broadcasts their public key. Then each party uses their own private key and the other party's public key to generate a shared secret key. DH employs the principles of modular arithmetic to ensure both parties end up with the same shared secret even though each party has access only to the other party's public key.

Modern cryptosystems based on [elliptic curve cryptography][33] (ECC) extend this concept with the [elliptic curve Diffie-Hellman)][63] (ECDH) key exchange. ECDH works similarly to DH but utilizes the properties of elliptic curves, resulting in a more secure and efficient system.

![Figure 2. Key Exchange Protocol](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/kep_akc.avif)

*Figure 2. Key Exchange Protocol*

[62]: #definition-tooltip "Reference: Diffie–Hellman key exchange,Wikipedia. Jul. 2023. Accessed: Jul. 10,2023. [link](https://en.wikipedia.org/w/index.php?title=Diffie%E2%80%93Hellman_key_exchange&oldid=1163912552)"
[63]: #definition-tooltip "Reference: Elliptic-curve Diffie–Hellman,Wikipedia.Jun. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic-curve_Diffie%E2%80%93Hellman&oldid=1160050860)"
[33]: #definition-tooltip "Reference: Elliptic-curve cryptography,Wikipedia. Jun. 2023. Accessed: Jun. 20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic-curve_cryptography&oldid=1160709993)"

### Non-interactive key exchange

Unlike key exchange protocols like DH and ECDH, which are interactive, requiring back-and-forth communication to decide the symmetric key, AKC also provides for non-interactive ways to establish a shared secret key. In such schemes, one party generates a pair of keys (public and private) and shares the public key with the other party. This second party then generates a random symmetric key, encrypts it with the received public key, and sends it back to the first party. The first party uses their private key to decrypt the received message, thereby obtaining the shared symmetric key. This scheme is non-interactive in that the symmetric key is determined by one party and simply communicated securely to the other party in encrypted form.

An important consideration in non-interactive key exchange has to do with the length differential in bits between the symmetric key the parties wish to exchange and recommended message sizes in AKC. Typically, modern symmetric keys are in the range of 128-256 bits long, while asymmetric key cryptosystems such as RSA work with message sizes around 1024-4096 bits long. Therefore, when using AKC to transmit a symmetric key, for security it must nevertheless be encoded into a longer 1024-4096 bit message. This can be achieved via two approaches:

- **Padding-based key exchange:** In this approach, the shorter (128-256 bit) symmetric key is generated first and then an agreed-upon reversible padding scheme such as [OAEP][71] is used to embed it into a longer (1024-4096 bit) message. This longer message is encrypted using AKC and broadcasted as [ciphertext][ciphertext]. The recipient first decrypts the ciphertext and then removes the padding to extract the shorter symmetric key.

[ciphertext]: #definition-tooltip "The unreadable output of the encryption algorithm."

- **Key [encapsulation][Encapsulation] mechanisms (KEMs):** With [KEM-based][61] key exchange, a random, long (1024-4096 bit) plain text message is first generated, from which a shorter (128-256 bit) symmetric key can be extracted using an agreed-upon [key derivation function][101] (KDF). The longer plain text is encrypted using AKC and broadcasted to the recipient as ciphertext. The recipient decodes the ciphertext using their private key and then uses the KDF to extract the shorter (128-256 bit) symmetric key. Popular cryptosystems such as [RSA][66], with their ability to directly encrypt data, can be used to implement KEMs.

![Figure 3. Key Encapsulation Mechanism](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/kem_akc.avif)

*Figure 3. Key Encapsulation Mechanism*

[rsa]: #definition-tooltip "A public-key cryptosystem named after Ron Rivest, Adi Shamir, and Leonard Adleman. Developed in 1977."
[61]: #definition-tooltip "Reference: Key encapsulation mechanism,Wikipedia. Sep. 2022. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Key_encapsulation_mechanism&oldid=1110687965)"
[71]: #definition-tooltip "Reference: Optimal asymmetric encryption padding,Wikipedia. Jul. 2023. Accessed: Jul. 11, 2023. [link](https://en.wikipedia.org/w/index.php?title=Optimal_asymmetric_encryption_padding&oldid=1163481577)"
[66]: #definition-tooltip "Reference: RSA(cryptosystem),Wikipedia. Jul. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=RSA_(cryptosystem)&oldid=1163962255)"
[Encapsulation]: #definition-tooltip "The process of combining data, such as a message or a file, with cryptographic techniques to create a secure unit; the goal is to predict integrity of the data while it is being stored or transmitted."
[101]: #definition-tooltip "Reference: Key derivation function,Wikipedia. Jul. 2023. Accessed: Aug. 04, 2023. [link](https://en.wikipedia.org/w/index.php?title=Key_derivation_function&oldid=1165650237)"

## Digital signatures with asymmetric key cryptography

[*Digital signatures*][49] are another powerful application of asymmetric key cryptography. They provide authentication, integrity, and nonrepudiation enabled by the fact that within AKC, entities possess unique private keys. The basic idea underlying signature protocols is that senders of secure messages will additionally *digitally sign* the messages using their unique private key. The receiver will then verify the digital signature using the public key of the sender. Within AKC, digital signatures can be implemented by using algorithms specifically designed for that purpose or by using generic cryptosystems.

![Figure 4. Digital signatures with asymmetric key cryptography](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/dig_sig_akc.avif)

*Figure 4. Digital signatures  with asymmetric key cryptography*

### Dedicated digital signature algorithms

Currently, the US Federal Information Processing Standard (FIPS) for digital signatures is a dedicated scheme simply titled the [*Digital Signature Algorithm*][28] (DSA). Somewhat similarly to the Diffie-Hellman protocol, the DSA uses the algebraic properties of the [modular exponential][modexp] and [multiplicative inverses][Multiplicative] for signature generation and verification.

The [elliptic curve digital signature algorithm][64] (ECDSA) is an ECC variant of DSA, providing the same functionality but with significantly shorter keys. This results in improved efficiency, making it a popular choice for systems with [resource constraints][65].

Both DSA and ECDSA will be illustrated in more detail later.

### Digital signature schemes using generic cryptosystems

In addition to dedicated algorithms, digital signatures can also be generated using generic asymmetric cryptosystems, such as [RSA][66].

RSA, which will be discussed in detail in a later section, also exploits modular multiplicative inverses and [modular exponentiation][Modular] as fundamental operations but combines them in a different sequence than DSA. In RSA, the signer typically creates a hash of the message and then encrypts the hash with their private key, creating the digital signature. Any party can then verify this signature by decrypting it with the signer's public key and comparing it with the hashed message.

[modexp]: #definition-tooltip "modular exponentiation is the remainder when an integer is raised to a power (exponent) and then divided by a number (modulus). For example $x^e mod m$ ."
[Modular]: #definition-tooltip "Used to efficiently compute large powers of a number modulo another number; by repeatedly reducing the result modulo  during the computation, modular exponentiation prevents the intermediate results from becoming extremely large, which can help in managing and performing calculations with large numbers efficiently."
[Multiplicative]: #definition-tooltip "The set of integers modulo a prime number, where the group operation is modular multiplication. Elements in the multiplicative group are chosen in such a way that they are coprime to the modulus, ensuring that each element has a unique multiplicative inverse within the group."
[rsa]: #definition-tooltip "A public-key cryptosystem named after Ron Rivest, Adi Shamir, and Leonard Adleman. Developed in 1977."
[66]: #definition-tooltip "Reference: RSA(cryptosystem),Wikipedia. Jul. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=RSA_(cryptosystem)&oldid=1163962255)"
[28]: #definition-tooltip "Reference: Digital Signature Algorithm,Wikipedia. Apr. 2023. Accessed:Jun. 20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Digital_Signature_Algorithm&oldid=1148950522)"
[64]: #definition-tooltip "Reference: Elliptic Curve Digital Signature Algorithm,Wikipedia. Jun. 2023. Accessed:Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic_Curve_Digital_Signature_Algorithm&oldid=1162097661)"
[49]: #definition-tooltip "Reference: Digital signature,Wikipedia. Jun. 2023. Accessed: Jun. 27, 2023. [link](https://en.wikipedia.org/w/index.php?title=Digital_signature&oldid=1161173191)"
[65]: #definition-tooltip "Reference: C.A. Lara-Nino, A. Diaz-Perez, and M. Morales-Sandoval,“Elliptic Curve Lightweight Cryptography: A Survey,”IEEE Access, vol.6, pp. 72514–72550, 2018, [link](https://doi.org/10.1109/ACCESS.2018.2881444)"

## Applications of asymmetric key cryptography

[Asymmetric key cryptography][59] is [ubiquitous][Ubiquitous] in modern digital technology applications. The basic functionalities of AKC described above form the building blocks of many higher-level application protocols, including:

1. **Internet communication:** Secure communication over the internet, such as HTTPS, relies heavily on asymmetric key cryptography. The Transport Layer Security (TLS) and its predecessor, Secure Sockets Layer (SSL), use asymmetric key cryptography during the initial handshake process to establish a symmetric key, which is then used for the rest of the communication session.

2. **Authentication:** Asymmetric key cryptography is used to create digital signatures, allowing an entity to authenticate a digital document or message as coming from a specific sender. This is used in many scenarios, from verifying software updates to legally binding digital contracts.

3. **Email encryption:** Email encryption protocols such as PGP (Pretty Good Privacy) and its open-source alternative GPG (GNU Privacy Guard) use asymmetric key cryptography to ensure that only the intended recipient can read the email content.

4. **Secure shell (SSH):** SSH is a protocol for secure remote login and other secure network services over an insecure network. It uses asymmetric key cryptography to authenticate the server to the client and, optionally, the client to the server.

5. **VPN (virtual private network):** Asymmetric key encryption is used to establish secure connections in VPNs, ensuring secure communication over public networks.

6. **Blockchain and cryptocurrencies:** Blockchain technologies, including Bitcoin and Ethereum, use asymmetric key cryptography. For example, the ownership of Bitcoin is established through digital signatures using asymmetric key cryptography.

7. **Certificate authorities:** Asymmetric key cryptography is used by certificate authorities (CAs) to issue and sign digital certificates, which are used in TLS communication, code signing, email encryption, and more. A digital certificate binds a public key to a specific entity (for example, a person or server).

![Figure 5. Issuing and signing digital certificates using asymmetric key cryptography](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/ca_akc.avif)

*Figure 5. Issuing and signing digital certificates using asymmetric key cryptography*

[Ubiquitous]: #definition-tooltip "Something that is present or found everywhere, or something that is widespread and commonly encountered."
[59]: #definition-tooltip "Reference: Public-key cryptography,Wikipedia. Jul. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Public-key_cryptography&oldid=1163952916)"

## Security of asymmetric key cryptography

Several cryptographic notions come together to enable secure asymmetric key cryptography, including:

**Private key secrecy**: The most basic security requirement of AKC is that the private key remains secret. However, since the private key must be mathematically linked to the public key, private key secrecy also requires that it be computationally infeasible to derive the private key from knowledge of the public key. Key generation schemes within AKC rely on computationally hard mathematical problems to facilitate this requirement.

**Trapdoor functionality** In AKC, the encryption and decryption operations involve different complementary keys from a single-key pair. Ciphertext generated by encryption using one of the keys (for example the public key) must be inscrutable to third parties while being easily decryptable by the holder of the complementary key (in this case the private key). In other words, encryption should resemble a [*trapdoor one-way function*][67] such that third parties cannot invert the operation and recover the plain text but the private key provides a secret *trapdoor* that enables easy inversion. Popular AKC algorithms use modular exponentiation to set up trapdoor one-way function behavior.

**Randomness**: The key generation process should also exploit randomness to ensure that keys are unpredictable, as any pattern or predictability in key generation could be exploited by an attacker. Randomness is also used for padding during encryption to generate semantically secure ciphertexts and within digital signature schemes to produce unique signatures even when the same message is signed multiple times. For this reason, the use of strong random number generators is an important part of AKC.

**Large key size**: As in the case of SKC, large key sizes ensure protection against brute force attacks. However, since large key sizes also increase the computational cost of the encryption and decryption process, an optimal solution needs to balance security and efficiency considerations. The following table shows typical key sizes for various asymmetric key cryptography protocols and applications:

| Protocol | Typical Key Sizes (in bits)         | Application                    |
|----------|-------------------------------------|--------------------------------|
| RSA      | 1024 (deprecated), 2048, 3072, 4096 | Encryption, digital signatures |
| DSA      | 1024 (deprecated), 2048, 3072       | Digital signatures             |
| DH       | 2048, 3072, 4096                    | Key exchange                   |
| ECDH     | 224, 256, 384, 521                  | Key exchange                   |
| ECDSA    | 224, 256, 384, 521                  | Digital signatures             |

**Public key infrastructure**: In AKC, the private keys must be kept secret by the owners while the public keys are shared. There needs to be a secure mechanism to manage and distribute these public keys between users. *Public key infrastructure* (PKI) provides a way to do this using *digital certificates*. A digital certificate provides proof of identity of the owner of the public key and is issued by a trusted authority like a certificate authority(which is part of a PKI). Hence, PKI plays an integral part in the security of modern application using AKC by enabling large-scale key management (by creating, managing, distributing and revoking digital certificates).

### Security risks to asymmetric key cryptography
As outlined in the above table, modern asymmetric key algorithms such as RSA typically employ much larger key sizes than commonly used symmetric key counterparts such as AES-128. Even the ECC-based protocols (ECDH and ECDSA) that have smaller key sizes employ a minimum of at least 224 bits for keys. This in turn implies that a brute force attack involving an exhaustive search of the private key space to identify the correct key is computationally intractable in the foreseeable future. This remains true even if quantum computers were to be deployed for this task. Therefore, attacks against AKC usually focus on exploiting other potential weaknesses of specific cryptosystems. Some well-documented attack modes target:

1. **Algorithmic weakness** by using sophisticated mathematical and computational means to undermine the hardness assumptions used to formulate asymmetric key algorithms. For example, the security of RSA is predicated on the difficulty of factoring large prime numbers, and recent computational advances have [enabled successful factoring of 829-bit RSA keys][68]. Therefore, 1024-bit RSA is currently deprecated. As will be discussed later, the primary risk posed by quantum computers to AKC also falls into this category.

2. **Imperfect randomness**, which can lead to weakness in the key generation process. For example, if the random number generator used to create the keys is flawed and generates keys that are not truly random, an attacker might be able to predict the keys.

3. **Implementation flaws** such as errors in the implementation of cryptographic algorithms that inadvertently reveal information about the keys. For example, incorrect padding can potentially reveal details about keys.

4. **Side channels** that refer to information leakage from the physical system that performs the cryptography. Such leaks could be in the form of timing information, power consumption, or even sound, which can be exploited in a so-called [*side-channel attack*][69]. For example, analyzing how long cryptographic operations take to execute could reveal the number of '1's in a binary key. This seemingly innocent leakage significantly narrows the search space, unveiling potential solutions to problems that initially seemed insurmountable.

5. **Key exchange** by intercepting keys while they are being exchanged, such as within a [*man-in-the-middle* attack][70] (MITM). The DH protocol is susceptible to MITM attacks if additional authentication steps are not incorporated.

6. **Key storage** by aiming to steal keys from poorly secured storage. This includes physical attacks such as manipulation or theft of storage devices.

Securing asymmetric key cryptosystems against the variety of attacks that are possible is therefore a significant task involving mathematical, hardware, software, logistical, and legal considerations.

[67]: #definition-tooltip "Reference: Trapdoor function,”Wikipedia. Jul. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Trapdoor_function&oldid=1163913265)"
[70]: #definition-tooltip "Reference: Man-in-the-middle attack,”Wikipedia. Jul. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Man-in-the-middle_attack&oldid=1164741355)"
[69]: #definition-tooltip "Reference: Side-channel attack,”Wikipedia. Jun. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Side-channel_attack&oldid=1162235564)"
[68]: #definition-tooltip "Reference: RSA Factoring Challenge,”Wikipedia. Jul. 2023. Accessed:Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=RSA_Factoring_Challenge&oldid=1163481797)"

## RSA

The [RSA (Rivest-Shamir-Adleman)][7] algorithm is one of the first public key cryptosystems and is widely used for secure data transmission. It is a versatile cryptosystem in that it provides the necessary operations to enable encryption, decryption, digital signatures, and key exchange within a common framework.

In this section, we will illustrate asymmetric key cryptography (AKC) using RSA through simple examples.

We will use the standard scenario of two parties, Alice and Bob, who wish to communicate securely using AKC.

### The RSA algorithm

The basic RSA algorithm involves four operations: key generation, key distribution, encryption, and decryption:

[7]: #definition-tooltip "Reference: R.L. Rivest, A. Shamir, and L. Adleman,“A method for obtaining digital signatures and public-key cryptosystems,”Communications of the ACM, vol. 21, no. 2, pp. 120–126, Feb.1978,[link](https://dl.acm.org/doi/10.1145/359340.359342)"

1. **Key generation:**

   Public and private keys are generated based on mathematical principles around [prime numbers][prime], where calculating them is easy, but the reverse is hard.

   We'll refer to these:
    - Public key: $(e, n)$
    - Private key: $(d, n)$

   Note that $n$ is common to both public key and private key, and is known as the modulus. We will need to use this later.

   <details>

   <summary>

   Mathematical detail

   </summary>

   - Choose two distinct prime numbers, $p$ and $q$.
     - chosen at random (for security).
     - They should be similar in magnitude, but differ in length by a few digits, to make factoring harder.
     - Prime numbers can be efficiently chosen using a [primality test][primalitytest].
   - Compute $n = p*q$.
     - $n$ is the modulus for both the public and private keys.
   - Compute the [totient][totient] [$φ$][phi]$(n) = (p-1)*(q-1)$.
     - The totient is meant to be kept secret and typically discarded after key generation.
   - Choose an integer $e$ such that $1 < e < $[$φ$][phi]$(n)$ and [$gcd$][gcd]$(e, $[$φ$][phi]$(n)) = 1$.
     - i.e. $e$ and [$φ$][phi]$(n)$ should be [coprime][coprime].
     - This number $e$ forms the public key [exponent][exponent] and is typically chosen as a small number for computational efficiency.
     - The prime number $65537 = 2^{16} + 1$ is often used.
     - Compute $d$ to satisfy the [congruence relation][congrel] $d*e ≡ 1 ($[$mod$][modulus][$φ$][phi]$(n))$.
       - That is, $d$ is the [multiplicative inverse][multinv] of $e$ modulo [$φ$][phi]$(n)$.
       - This is more efficiently computed using the [extended Euclidean algorithm][exteuc].
       - This number $d$ is the private key [exponent][exponent].
   - The public key consists of $(e, n)$, and the private key is $(d, n)$.

   </details>

[coprime]: #definition-tooltip "Two numbers that have a greatest common divisor of 1. For example, 3 and 13 are coprime because only 1 can divide both of them."
[congrel]: #definition-tooltip "Consistency, or equivalence ie. all behaving in the same way."
[exteuc]: #definition-tooltip "An efficient way of calculating the greatest common divisor of two whole numbers. It improves on the original Euclidean algorithm."
[exponent]: #definition-tooltip "The number of times a number is multiplied by itself."
[gcd]: #definition-tooltip "Greatest Common Divisor - the largest positive whole number that can divide two or more of the given whole numbers."
[modulus]: #definition-tooltip "The modulus is the remainder left over after dividing by the value specified. modulo is the operation of doing this."
[multinv]: #definition-tooltip "Also known as reciprocal, a number, which when multiplied by the number given, results in a value of 1."
[phi]: #definition-tooltip "The greek symbol, phi, often used to indicate Euler's totient."
[primalitytest]: #definition-tooltip "An algorithm to determine whether a given number is prime. This is relatively easy computationally compared to factorization."
[prime]: #definition-tooltip "A positive, whole number, that can only be divided by 1 or itself."
[totient]: #definition-tooltip "The totient is a number, or numbers, that are not greater than the given number, and are relatively prime to it, having no factors in common."

2. **Key distribution:**

   - The public key $(e, n)$ is made public to those who may wish to send a message
   - The private key $(d, n)$ is kept secret.

3. **Encryption:**

   - Alice wishes to send a message $M$ to Bob. In this case a simple integer
   - Alice uses Bob's public key $(e, n)$ to encrypt the message into ciphertext $C$


   <details>

   <summary>

   Mathematical detail

   </summary>

   - $M$ is an [integer][intrep] $0 ≤ M < n$.
   - $C ≡ M^e (mod n)$, where $C$ is the ciphertext.

   </details>

   [intrep]: #definition-tooltip "We consider an integer in this example as the simplest case. Any message can be converted into an integer based representation."

4. **Decryption:**

   - Bob receives the ciphertext $C$
   - Bob uses his private key $(d, n)$ to decrypt the message back into message $M$


   <details>

   <summary>

   Mathematical detail

   </summary>

   - $M ≡ C^d (mod n)$.

   </details>

This is the basic outline of RSA. In practice, [more sophisticated padding schemes][71] are applied to the plain text $M$ before encryption to ensure that equal plain texts result in different ciphertexts. This prevents a range of possible attacks against naive implementations of RSA and enables [semantic security][semsec].

[semsec]: #definition-tooltip "With only the ciphertext, and any relevant public information (such as public key for asymmetric key cryptography), and adversary cannot retrieve even partial information from the plaintext."
[71]: #definition-tooltip "Reference: Optimal asymmetric encryption padding,Wikipedia. Jul. 2023. Accessed: Jul. 11, 2023. [link](https://en.wikipedia.org/w/index.php?title=Optimal_asymmetric_encryption_padding&oldid=1163481577)"

### Illustration of RSA in Python

In the following code cells, we illustrate a simple example of the RSA algorithm using small integers and then demonstrate practical key distribution and digital signature applications using Python libraries implementing RSA.

> Note: In this section we will show the math calculations in detail as part of the Python code

#### RSA key generation

Let's step through a simple instance of the RSA algorithm employing small prime numbers.

We will need to be able to compute the [greatest common divisor][gcd] of two integers, as it will be needed to test whether two integers are [coprime][Coprime].

We will explain one simple way to calculate this, but it is much more efficient with larger integers to use the `math.gcd` python function.

[coprime]: #definition-tooltip "Two numbers that have a greatest common divisor of 1. For example, 3 and 13 are coprime because only 1 can divide both of them."
[gcd]: #definition-tooltip "Greatest Common Divisor - the largest positive whole number that can divide two or more of the given whole numbers."

In [ ]:
import math


# Example function to compute the gcd (greatest common divisor)
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)


# let's calculate some examples using algorithm
n1 = gcd(50, 10)
n2 = gcd(99, 33)
n3 = gcd(59, 9)

# do the same with the python library call

m1 = math.gcd(50, 10)
m2 = math.gcd(99, 33)
m3 = math.gcd(59, 9)

# Confirm they are the same
assert n1 == m1
assert n2 == m2
assert n3 == m3

# They are - print out the values for explanation
print("gcd(50,10) =", m1)
print("gcd(99,33) =", m2)
print("gcd(59,9) =", m3)

The first phase of the RSA workflow is key generation. This is initiated by choosing two prime numbers, which are meant to be kept secret by the entity generating the keys.

In [ ]:
# Choosing two prime numbers and keep them secret
p = 13
q = 19
print("The secret prime numbers p and q are:", p, q)

Next, the [modulus][modulus] $n$, which is simply the product of the two chosen primes, is computed. This modulus will be published as part of the public key.

[modulus]: #definition-tooltip "The modulus is the remainder left over after dividing by the value specified. modulo is the operation of doing this."

In [ ]:
# Calculate n which is the modulus for both the public and private keys
n = p * q
print("modulus n (p*q)=", n)

The Euler [totient][totient] function [$\varphi(n)$][phi] is computed next, as it is needed for the [modular multiplicative inverse][modinv] operation used to determine the keys in RSA. $phi$ is also kept secret and typically discarded after key generation.

[modinv]: #definition-tooltip "Given an integer $a$, and modulus $m$, the remainder from ax/m is 1. This is only the case for coprimes."
[phi]: #definition-tooltip "The greek symbol, phi, often used to indicate Euler's totient."
[totient]: #definition-tooltip "The totient is a number, or numbers, that are not greater than the given number, and are relatively prime to it, having no factors in common."

In [ ]:
# Compute Euler's totient function, φ(n) and keep it secret
phi = (p - 1) * (q - 1)
print("The secret Euler's function (totient) [phi(n)]:", phi)

We are now ready to calculate the public and private keys. In RSA, each of these is specified by a tuple of two integers. The first entry in each tuple is a distinct integer, and the second entry is the modulus $n$ that is common to both keys.

The first entry in the public key can be any integer greater than 1 that is coprime to $phi$. Two integers are coprime if their [greatest common divisor][gcd] is 1. So we use the `math.gcd` function find an integer $e$ coprime to $phi$.

[gcd]: #definition-tooltip "Greatest Common Divisor - the largest positive whole number that can divide two or more of the given whole numbers."

In [ ]:
# Choose an integer e such that e and φ(n) are coprime
e = 2
while e < phi:
    if math.gcd(e, phi) == 1:
        break
    else:
        e += 1
print("Public Key (e):", e)

The private key requires an integer $d$, which is the [multiplicative inverse][multinv] of $e$ modulo $phi$; that is, it satisfies the [congruency relation][congrel] $d*e\equiv 1 \pmod{\varphi(n)}$. For this simple illustration where we are dealing with small integers, we can just loop over the positive integers to locate a suitable $d$. In realistic settings, the computationally efficient [extended Euclidean algorithm][exteuc] is used for this purpose.

[congrel]: #definition-tooltip "Consistency, or equivalence ie. all behaving in the same way."
[exteuc]: #definition-tooltip "An efficient way of calculating the greatest common divisor of two whole numbers. It improves on the original Euclidean algorithm."
[multinv]: #definition-tooltip "Also known as reciprocal, a number, which when multiplied by the number given, results in a value of 1."

In [ ]:
# Compute a value for d such that (d * e) % φ(n) = 1
d = 1
while True:
    if (d * e) % phi == 1:
        break
    else:
        d += 1
print("Private Key (d):", d)

We now form the tuples $(e, n), (d, n)$, which form the public and private keys respectively. The public key is then published, while the private key is kept secret.

In [ ]:
# Public and Private Key pair
public = (e, n)
private = (d, n)

print(f"The Public key is {public} and Private Key is {private}")

Encryption and decryption in RSA use the [modular exponentiation operation][72]. Further, the public and private keys are complementary in that either can be used to encrypt a message that the other can then decrypt.

Here we illustrate the case where the public key $(e,n)$ is used for encryption and the private key $(d, n)$ is used for decryption by defining a Python function for each operation.

We then encrypt and decrypt an integer message $msg$.

[72]: #definition-tooltip "Reference: Modular exponentiation,”Wikipedia. Jun. 2023. Accessed: Jul. 11, 2023. [link](https://en.wikipedia.org/w/index.php?title=Modular_exponentiation&oldid=1160764843)"

In [ ]:
# Encryption function
def encrypt(plain_text):
    return (plain_text**e) % n


# Decryption function
def decrypt(cipher_text):
    return (cipher_text**d) % n


# Simple message to encode
msg = 9

# encrypt then decrypt
enc_msg = encrypt(msg)
dec_msg = decrypt(enc_msg)

print("Original Message:", msg)
print("Encrypted Message:", enc_msg)
print("Decrypted Message:", dec_msg)

While the small integers used above are useful to easily outline the core ideas in the RSA algorithm, in real applications RSA requires the use of very large integers. For example, 2048-bit RSA involves the use of a [modulus][modulus] $n$ that is 2048 bits long, the decimal integer equivalents of which are around 10$^616$. These truly enormous numbers are necessary for the practical security of RSA.

[modulus]: #definition-tooltip "The modulus is the remainder left over after dividing by the value specified. modulo is the operation of doing this."

#### Symmetric key exchange with RSA
As discussed previously, [AKC][akc] enables two parties who wish to communicate to securely establish a *shared secret*, which can be used, for instance, as the secret key for subsequent symmetric encryption of bulk plain text.

[akc]: #definition-tooltip "Asynchronous key encryption."

Let us consider the following scenario. Alice and Bob want to use [SKC][skc] to encrypt and decrypt messages. However, before this process can be initialized, they need to agree on a common secret key. One option is for one party — say, Alice — to generate a secret key and then securely transmit it to Bob. To achieve this secure transfer, Alice decides to use RSA as the [key encapsulation mechanism][kem] (KEM).

This involves the following steps:

- First, Alice generates a random symmetric key, which she intends to share with Bob.
- Then, Bob generates an asymmetric key pair and makes his public key available on a suitable channel.
- Next, Alice uses Bob's public key to encrypt the symmetric key, thus encapsulating it in a ciphertext.
- Then, Alice broadcasts the ciphertext over a reliable but not necessarily secure channel.
- Finally, Bob receives the ciphertext and decrypts it using his private key. He now has access to the symmetric key generated by Alice.

![Figure 1. Symmetric key exchange with RSA](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/key_exchange_rsa.avif)

*Figure 1. Symmetric key exchange with RSA*

##### Padding-based key-exchange example in Python
A practical workflow utilizing RSA for padding-based non-interactive key exchange is now illustrated using the ``cryptography`` Python library.

Import necessary modules from the ``cryptography`` Python library. If needed, this library can be installed using the command ``pip install cryptography``.

Alice then generates a random secret key, which she intends to transmit to Bob.

[kem]: #definition-tooltip "Key encapsulation mechanism. Secures a symmetric key by use of asymmetric key encryption. A simple approach is to encrypt a random symmetric key using a public key, but more advanced & secure kems are available, and new ones continually developed."
[skc]: #definition-tooltip "Symmetric key encryption."

In [ ]:
# pip install cryptography
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes

symmetric_key = Fernet.generate_key()
print(f"\nSymmetric key generated by Alice: {symmetric_key}")

Using the ``rsa`` module from the ``cryptography`` library, Bob generates a key pair and then broadcasts his public key. Any one can intercept the public key and read off the public numbers $(e,n)$ that form the key.

In [ ]:
# Bob generates a 2048-bit RSA key pair
bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()
print(f"Public key broadcast by Bob: {bob_public_key}")
print(f"\nPublic numbers in Bobs' public key: {bob_public_key.public_numbers()}")

At this point, we assume Alice has received the public key broadcast by Bob. Alice's ``symmetric_key`` above can now be encrypted using Bob's public key to produce the ciphertext. In a realistic setting, Alice will also use additional [padding][padding] methods such as [OAEP][71] to ensure [semantic security][semsec] for her communications with Bob.

[padding]: #definition-tooltip "The addition of data to the beginning, middle, or end of a message. Often used in block ciphers to fill out blocks with insufficient data up to the required size."
[semsec]: #definition-tooltip "With only the ciphertext, and any relevant public information (such as public key for asymmetric key cryptography), and adversary cannot retrieve even partial information from the plaintext."
[71]: #definition-tooltip "Reference: Optimal asymmetric encryption padding,Wikipedia. Jul. 2023. Accessed: Jul. 11, 2023. [link](https://en.wikipedia.org/w/index.php?title=Optimal_asymmetric_encryption_padding&oldid=1163481577)"

In [ ]:
# Encryption
ciphertext = bob_public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)

print("Ciphertext:", ciphertext)

Alice then broadcasts the ciphertext over an open channel, confident that only Bob with the corresponding private key will be able to decrypt it. We assume Bob has received the ciphertext and can now decrypt it using his confidential private key.

In [ ]:
# Bob decrypts ciphertext to access the symmetric key
decrypted_symmetric_key = bob_private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)

print("Decrypted key:", decrypted_symmetric_key)
assert decrypted_symmetric_key == symmetric_key

At this point, Alice and Bob both have access to the secret symmetric key, which they can use for [SKC][skc] applications.

[skc]: #definition-tooltip "Symmetric key encryption."

##### Simulating a Key Encapsulation Mechanism with RSA in Python
In the following workflow, we illustrate the use of RSA to simulate a Key Encapsulation Mechanism (KEM) whereby a sufficiently long random secret message is securely exchanged and subsequently converted into a shared-secret of the appropriate length using a KDF.

Once again Alice and Bob want to establish a shared secret non-interactively and Alice is the party that decides which secret to use.

We start by importing some necessary Python libraries.

Bob then generates his RSA key pair and broadcasts his public key.

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from os import urandom

# Bob's RSA key pair
private_key_Bob = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key_Bob = private_key_Bob.public_key()

print("Bob's private and public keys created")

Alice first generates a long random secret from which the shared secret will be eventually derived. In a pure [KEM][61], the long secret will be a random element from the algebraic structure underlying the cryptosystem. In the case of 2048-bit RSA, this would be a random integer modulo the 2048-bit RSA modulus. As such a pure KEM does not require additional padding but in this example we are only simulating a KEM using RSA and the `cryptography` library requires the use of padding when encrypting with RSA. So we will use a somewhat shorter long secret which nevertheless is much longer than a standard 256-bit AES key.

[61]: #definition-tooltip "Reference: Key encapsulation mechanism,Wikipedia. Sep. 2022. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Key_encapsulation_mechanism&oldid=1110687965)"

In [ ]:
Alice_long_secret = urandom(160)  # A 160 byte or 1280 bit random message
print("Alice's secret created")

Next Alice encrypts the long secret using Bob's public key and the encrypted secret is communicated to Bob.

In [ ]:
Alice_encrypted_secret = public_key_Bob.encrypt(
    Alice_long_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)
print("Alice's secret encrypted")

Bob decrypt's the encrypted secret received from Alice using his private key.

In [ ]:
Bob_decrypted_secret = private_key_Bob.decrypt(
    Alice_encrypted_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)

assert Alice_long_secret == Bob_decrypted_secret, "Secrets do not match!"

# if we get here they match
print("Secrets match")

Finally both Alice and Bob separately apply an agreed upon [Key Derivation Function][101] (KDF) on the long secret to derive the symmetric key.

Note that the process involves a hashing protocol and the use of a random [*salt*][salt] which ensures uniqueness and unpredictability of the shared symmetric key in case the long secret is ever reused (not recommended). However the *salt* itself does not have to be secret and once it is randomly generated, say by Alice in this example, it can be broadcasted to Bob alongside the encrypted long secret.

We will assume therefore that both Alice and Bob have access to the same random *salt*.

[salt]: #definition-tooltip "Regarding password hashing,the idea that salt enhances/makes a food unique is the same idea of how adding salt in this context makes our password unique, distinctive, and hard to reproduce."
[101]: #definition-tooltip "Reference: Key derivation function,Wikipedia. Jul. 2023. Accessed: Aug. 04, 2023. [link](https://en.wikipedia.org/w/index.php?title=Key_derivation_function&oldid=1165650237)"

In [ ]:
def key_derivation_function(secret, salt):
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,  # Desired key length
        salt=salt,
        info=None,
        backend=None,
    )
    return hkdf.derive(secret)


common_salt = urandom(16)  # Random salt accessible to both Alice and Bob

symmetric_key_Alice = key_derivation_function(Alice_long_secret, common_salt)
symmetric_key_Bob = key_derivation_function(Bob_decrypted_secret, common_salt)

assert symmetric_key_Alice == symmetric_key_Bob, "Derived keys do not match!"
print(
    f"A symmetric key of length {len(symmetric_key_Alice)*8} bits was successfully derived by both Alice and Bob!"
)

#### Digital signatures with RSA
We will now extend the above confidential communication scenario with Alice and Bob to one that also includes validation with the help of a [digital signature][49].

As before, Alice will confidentially send a message encapsulating a symmetric key to Bob, but she will also digitally sign the message so that Bob, upon receiving the message, can verify that it was Alice who originated it and that the contents of the message were not tampered with during transmission.

More generally, it is desirable to enable validation without compromising confidentiality whereby any interested party is able to verify the [integrity][integrity], [authenticity][authenticity], and establish non-repudiation with respect to a given communication, even if that party does not have access to the actual plain text message.

We will consider this general setting which then involves the following steps:

- First, both Bob and Alice make their public keys available over an open channel.
- Then, Alice encrypts the plain text using Bob's public key, creating a ciphertext.
- Next, Alice creates a hash of the ciphertext with a *hash function* and further encrypts the resulting hashed ciphertext using her private key. This encrypted hash is the *signature*.
- Then, Alice then transmits both the ciphertext and the signature over an open channel.
- Then, Bob uses Alice's public key to decrypt the signature, revealing the hashed ciphertext.
- Next, since Bob also has access to the ciphertext itself, he uses the same hash function used by Alice to recreate a second instance of the hashed ciphertext. If the latter matches the one obtained by decrypting Alice's signature, then the message is validated, even though the ciphertext itself has not yet been decrypted.
- Finally, Bob, having validated the message, decrypts the ciphertext using his own private key.

![Figure 2. Digital signatures with RSA](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/dig_sig_rsa.avif)

*Figure 2. Digital signatures with RSA*

This workflow for a digital signature is illustrated next.

We once again import some useful modules from the ``cryptography`` library.


[authenticity]: #definition-tooltip "Authenticity is the proof that a message is from the source it claims to be from."
[integrity]: #definition-tooltip "Integrity is maintained when a message is protected against unauthorized change."
[49]: #definition-tooltip "Reference: Digital signature,Wikipedia. Jun. 2023. Accessed: Jun. 27, 2023. [link](https://en.wikipedia.org/w/index.php?title=Digital_signature&oldid=1161173191)"

As before, Alice intends to securely send a symmetric key to Bob, but she also wishes to digitally sign it. In this case, we need public keys for both Alice and Bob. Therefore, the first step is for both Alice and Bob to create their own key pair using RSA and broadcast their own public key to the world.

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed

# Generate keys for Bob
bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()

# Generate keys for Alice
alice_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
alice_public_key = alice_private_key.public_key()

print("Private and Public keys generated for Bob and Alice.")

In the next step, as before, Alice uses Bob's public key to encrypt the symmetric key and prepares the ciphertext.

In [ ]:
# Alice encrypts the message using Bob's public key
ciphertext = bob_public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)

print("ciphertext of symmetric key: ", ciphertext)

At this stage, instead of just broadcasting the ciphertext, Alice intends to attach a digital signature to it so that she can prove to Bob that she was the sender of the message. This is done in two steps:

1. Create a hash of the ciphertext using a hashing algorithm.
2. Encrypt the hash using Alice's private key, which amounts to a signature.

In [ ]:
# Alice signs the ciphertext using her private key
digest = hashes.Hash(hashes.SHA256())
digest.update(ciphertext)
hash_to_sign = digest.finalize()

signature = alice_private_key.sign(
    hash_to_sign,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    Prehashed(hashes.SHA256()),
)

print("signature: ", signature)

Alice then broadcasts the ciphertext and the signature over a network so that Bob is able to intercept both of them.

In [ ]:
# Bob receives the ciphertext and signature
received_ciphertext = ciphertext
received_signature = signature

# Send signature and ciphertext here
print("Sending ciphertext and signature.....")

On Bob's side, the first task is to verify the [integrity][integrity] and [authenticity][authenticity] of the ciphertext. To do this, Bob creates a hash of the received ciphertext using the same hashing algorithm used by Alice.

[authenticity]: #definition-tooltip "Authenticity is the proof that a message is from the source it claims to be from."
[integrity]: #definition-tooltip "Integrity is maintained when a message is protected against unauthorized change."

In [ ]:
# Bob creates a hash of the ciphertext using the same algorithm used by Alice
digest = hashes.Hash(hashes.SHA256())
digest.update(received_ciphertext)
hash_to_verify = digest.finalize()

print("hash to verify: ", hash_to_verify)

Then, Bob decrypts the received signature using Alice's public key. Because Alice used her private key to create the signature, Bob is able to decrypt it using Alice's public key. The decrypted signature is nothing but a hash of the ciphertext created on Alice's end. If the hash created by Bob matches the decrypted signature, then Bob has verified that the ciphertext he received has not been tampered with and that it was Alice who signed the ciphertext.

In the Python code below, these operations are combined into a useful utility function called ``verify`` provided by an object associated with Alice's public key.

In [ ]:
from cryptography.exceptions import InvalidSignature


def is_signature_valid(public_key, signature, data_hash):
    try:
        public_key.verify(
            signature,
            data_hash,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH
            ),
            Prehashed(hashes.SHA256()),
        )
        return True
    except InvalidSignature:
        return False


if is_signature_valid(alice_public_key, received_signature, hash_to_verify):
    print("The signature is valid.")
else:
    print("The signature is not valid.")

Having verified the [integrity][integrity] and [authenticity][authenticity] of the received ciphertext, Bob can then decrypt it using his private key because Alice created the ciphertext using Bob's public key.

[authenticity]: #definition-tooltip "Authenticity is the proof that a message is from the source it claims to be from."
[integrity]: #definition-tooltip "Integrity is maintained when a message is protected against unauthorized change."

In [ ]:
# Bob decrypts the message using his private key
decrypted_message = bob_private_key.decrypt(
    received_ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)

print("Decrypted message:", decrypted_message.decode())

In the above digital signature scenario, any party — not just Bob — can verify that Alice is the sender of the ciphertext because everyone can access Alice's public key, the ciphertext, and the digital signature. Furthermore, after sending the ciphertext and the signature, Alice cannot later deny having done so because the signature can be decrypted to a meaningful hash using only her public key. This establishes [non-repudiation][nonrepudiation].

By enabling secure key distribution and supporting non-repudiation, public key cryptography establishes a strong foundation for secure digital communication.

[nonrepudiation]: #definition-tooltip "Non-repudiation refers to the ability to prove that a specific party has sent a message. In symmetric key cryptography, since the same key is used for both encryption and decryption, it is not possible to determine which party has created a particular ciphertext. In contrast, asymmetric key cryptography provides non-repudiation through the use of digital signatures."

## Breaking RSA

The utility and security of the [RSA][rsa] algorithm outlined above rests on two mathematical assumptions:

1. Finding the [modular multiplicative inverse][modinv] $d$ given that access only to $(e, n)$ is computationally infeasible.

2. In the RSA setting, the modular exponentiation operation behaves like a [one-way trapdoor function][trapdoor]. When used for encryption, it yields ciphertext that is inscrutable, and without access to the private key, inverting the operation to recover the plain text from the ciphertext is not feasible. However, with access to the private key, which acts as a trapdoor, the ciphertext can be easily decrypted.

The most prominent attack on the [RSA][rsa] algorithm aims to undermine assumption 1 by efficiently recovering the private key number $d$ via factorizing the [modulus][modulus] $n$. As will be illustrated below, it is easy to recover $d$ if one has access to either the [prime factors][primefactor] $p$ and $q$ of $n$ or the totient [$φ$][phi]$(n)$. Recall that $p$, $q$, and [$φ$][phi]$(n)$ are kept secret during key generation and discarded after. A third party that recovers this information using a classical or quantum computer essentially uncovers the private key, breaking RSA. Thus, prime factorization is the key computational primitive necessary for breaking RSA.

[modinv]: #definition-tooltip "Given an integer $a$, and modulus $m$, the remainder from ax/m is 1. This is only the case for coprimes."
[modulus]: #definition-tooltip "The modulus is the remainder left over after dividing by the value specified. modulo is the operation of doing this."
[phi]: #definition-tooltip "The greek symbol, phi, often used to indicate Euler's totient."
[primefactor]: #definition-tooltip "prime factors of a number n, are those primes that are multiplied together to make n."
[rsa]: #definition-tooltip "A public-key cryptosystem named after Ron Rivest, Adi Shamir, and Leonard Adleman. Developed in 1977."
[trapdoor]: #definition-tooltip "A one-way function where it is easy to compute an output from the inputs, but hard to determine the inputs from the output."

### Classical computing and RSA

Prime factorization of large integers is known to exhibit [super-polynomial][superpoly] or [subexponential][subexp] scaling on classical computers. The best-known classical algorithm for factoring integers larger than $10^{100}$ is the *[general number field sieve][gnfs]* [(GNFS)][73].

<details>

<summary>

Mathematical detail

</summary>

$L_n[\frac{1}{3}, (\frac{64}{9})^{(\frac{1}{3})}] = e^{[((\frac{64}{9})^{(\frac{1}{3})} + o(1)) (log(n))^{\frac{1}{3}}(log(log(n))^{\frac{2}{3}})]}$

as a function of the integer $n$ to be factorized.

This scaling is super-polynomial in the number of bits required to represent $n$.

Therefore, prime factorization is considered inefficient on classical computers.

</details>

At present, the [largest integers factored on classical hardware][68] are in the range of 829 bits or 250 decimal digits. Given the exponential growth in classical computing power that has been witnessed over the last several decades, 1024-bit RSA is no longer considered secure in the near term and is now deprecated. Nevertheless, in the foreseeable future, factoring 2048-bit integers whose magnitude are in the range of $10^{617}$ is at present considered infeasible on classical systems, suggesting continued viability. The advent of quantum computers, however, voids this assessment.

[subexp]: #definition-tooltip "Scaling slower than exponential. For example polynomial and linear scaling."
[superpoly]: #definition-tooltip "growing faster than a polynomial, for example an exponential function."
[68]: #definition-tooltip "Reference: RSA Factoring Challenge,”Wikipedia. Jul. 2023. Accessed:Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=RSA_Factoring_Challenge&oldid=1163481797)"
[73]: #definition-tooltip "Reference: “General number field sieve,”Wikipedia. Apr. 2023. Accessed: Jul. 11, 2023. [Online].Available:[link](https://en.wikipedia.org/w/index.php?title=General_number_field_sieve&oldid=1151781536)"
[gnfs]: #definition-tooltip "General Number Field Sieve - An efficient classical algorithm for factoring large integers."

### Shor's quantum algorithm and RSA

Probably the most well-known quantum algorithm today is [Shor's algorithm][shor] for finding the prime factors of integers. When [introduced by Peter Shor in 1994][6], it was recognized as the first quantum algorithm that offered a [super-polynomial speedup][superpoly] relative to classical algorithms on a problem of great practical importance, namely prime factorization.

Shor's algorithm can factor primes with $O(n^2)$ where $n$ is the number of bits.

[shor]: #definition-tooltip "A quantum algorithm which can find prime factors efficiently. Developed by the mathematician, Peter Shor, in 1994."
[superpoly]: #definition-tooltip "growing faster than a polynomial, for example an exponential function."
[6]: #definition-tooltip "Reference: P.W. Shor,“Algorithms for quantum computation: Discrete logarithms and factoring,”in Proceedings 35th Annual Symposium on Foundations of Computer Science, Nov. 1994, pp. 124–134. [link](https://doi.org/10.1109/SFCS.1994.365700)."

<details>

<summary>

Mathematical explanation of Shor's algorithm

</summary>

In the context of [RSA][rsa], [Shor's algorithm][shor] works by exploiting the [periodicity][period] of the [modular exponential function][modexp] $f(x) = a^x (mod~n)$ and provides a quantum [period][period]-finding primitive that enables efficient prime factorization of the [modulus][modulus] $n$.

A simplified high-level outline of Shor's overall scheme for breaking RSA is as follows:

1. Given the modulus $n$, which is published as part of the public key, choose a number $a$ coprime to $n$ i.e., $gcd(a,n) = 1$. Since we know that $n = p*q$ has exactly two prime factors $(p, q)$, almost any number less than $n$ that we pick at random is likely to be coprime to $n$.

2. Having chosen $a$, find the exponent $r$ such that $a^r \equiv 1 (mod~n)$. This implies $a^r - 1 \equiv 0 (mod~n)$. The existence of an [exponent][exponent] $r$ such that the above [congruence][congrel] holds is guaranteed by the [periodicity property][period] of [modular exponentiation][modexp].

3. If $r$ is even, $a^r - 1 \equiv 0 (mod~n) \implies (a^{r/2} - 1) (a^{r/2} + 1) =  \gamma*n$ for some integer $\gamma$. The left-hand side of this latter equality must contain $p$ and $q$ as two of its prime factors since the right-hand side does. If $r$ is odd, go back to step 1 and try a different choice for $a$.

4. Use the [extended Euclid algorithm][exteuc] for finding $gcd((a^{r/2} - 1), n)$ or $gcd((a^{r/2} + 1), n)$. The computed [GCD][gcd] is very likely to identify one of the prime factors $p$ or $q$. Divide $n$ with this prime factor to recover the other.

5. Once $p, q$ are known, use the steps from the original RSA algorithm for reconstructing the totient $\phi(n)$ and generating the private key number $d$ as the [modular inverse][modinv] of the known public key number $e$.

</details>

[congrel]: #definition-tooltip "Consistency, or equivalence ie. all behaving in the same way."
[exponent]: #definition-tooltip "The number of times a number is multiplied by itself."
[exteuc]: #definition-tooltip "An efficient way of calculating the greatest common divisor of two whole numbers. It improves on the original Euclidean algorithm."
[gcd]: #definition-tooltip "Greatest Common Divisor - the largest positive whole number that can divide two or more of the given whole numbers."
[modexp]: #definition-tooltip "modular exponentiation is the remainder when an integer is raised to a power (exponent) and then divided by a number (modulus). For example $x^e mod m$ ."
[modinv]: #definition-tooltip "Given an integer $a$, and modulus $m$, the remainder from ax/m is 1. This is only the case for coprimes."
[modulus]: #definition-tooltip "The modulus is the remainder left over after dividing by the value specified. modulo is the operation of doing this."
[period]: #definition-tooltip "The interval at which a function repeats itself."
[rsa]: #definition-tooltip "A public-key cryptosystem named after Ron Rivest, Adi Shamir, and Leonard Adleman. Developed in 1977."
[shor]: #definition-tooltip "A quantum algorithm which can find prime factors efficiently. Developed by the mathematician, Peter Shor, in 1994."

In August 2023 Oded Regev [published an improvement][301] on Shor's original using a multi-dimensional approach resulting in $O(n^{1.5})$. There continues to be further research including by [Ragavan and Vaikuntanathan][302] in this area which may improve time, cost, or number of qubits needed. Whilst we cannot say when running such algorithms against real-world RSA encryption truly becomes viable, it is becoming closer all the time.

[301]: #definition-tooltip "Reference: Oded Regev,“An Efficient Quantum Factoring Algorithm.” arXiv, Aug 2023. [link](https://doi.org/10.48550/arXiv.2308.06572)"
[302]: #definition-tooltip "Reference: Seyoon Ragavanm and Vinod Vaikuntanathan,“Optimizing Space in Regev’s Factoring Algorithm.” arXiv, Oct 2023. [link](https://doi.org/10.48550/arXiv.2310.00899)"

#### Python example demonstrating breaking RSA encryption

In the following code cells, we illustrate an example of finding a private key given only the public key. This will use brute-force classical computation, but shows how Shor's algorithm could be used - including large keys.

> Note: In this section we will show the math calculations in detail as part of the Python code

In the example, we have a public key $(5, 247)$, and we will recover the private key.

**Step 1:** The first step is to pick a number coprime to 247. Almost any number we guess will do the job. Let's pick 6.

In [ ]:
n = 247  # the modulus
e = 5  # public key number
a = 6  # an integer coprime to n
assert gcd(a, n) == 1
print(f"Checked {n} and {a} are coprime")

**Step 2:** Next we need to find the [period][period] $r$ such that $6^r \equiv 1 (mod~247)$. In this example, we compute $r$ classically using brute force, but we could also use Shor's algorithm on a quantum computer using [Qiskit][qiskit].

[period]: #definition-tooltip "The interval at which a function repeats itself."
[qiskit]: #definition-tooltip "An open-source SDK for working with quantum algorithms and computers. Developed by IBM."

In [ ]:
r = 0
rem = 100
while rem != 1:
    r += 1
    rem = (a**r) % n

print(f"period r is: {r}")
assert a**r % n == 1

print(f"Checked {a}^{r} mod {n} is 1")

**Step 3:** Since the [period][period] $r = 36$ is even, we can compute $f1 = (a^{r/2}-1), f2=(a^{r/2}+1)$.

[period]: #definition-tooltip "The interval at which a function repeats itself."

In [ ]:
# explicitly use as integer
f1 = int(a ** (r / 2) - 1)
f2 = int(a ** (r / 2) + 1)

print(f"f1 = {f1}")
print(f"f2 = {f2}")

**Step 4:** Find the [GCD][gcd] of either of those factors with $n$. Simply divide $n$ with the prime factor already found to obtain the second prime factor.

[gcd]: #definition-tooltip "Greatest Common Divisor - the largest positive whole number that can divide two or more of the given whole numbers."

In [ ]:
q_found = gcd(f1, n)
print(f"One possible prime factor of n ({n}) is: {q_found}")

# explicit int (to avoid floating point)
p_found = int(n / q_found)
print(f"The second prime factor of n ({n}) is: {p_found}")

assert n == p_found * q_found

**Step 5:** Having recovered the prime factors of $n = 247$ as $p_{found}=13$ and $q_{found}=19$, we compute the totient $\phi_{found} = (p_{found}-1)*(q_{found}-1)$.

The private key is the [modular inverse][modinv] of the public key number $e=5$.

[modinv]: #definition-tooltip "Given an integer $a$, and modulus $m$, the remainder from ax/m is 1. This is only the case for coprimes."

In [ ]:
# Compute the totient
phi_found = (p_found - 1) * (q_found - 1)
print(f"The totient is: {phi_found}")

# Recover the private key number d_found by satisfying (d_found * e) % phi_found = 1
d_found = 1
while True:
    if (d_found * e) % phi_found == 1:
        break
    else:
        d_found += 1
print("Private Key number:", d_found)

In the above scheme, step 2 is the crucial [period][period]-finding operation for which [Shor's algorithm][76] uses two fundamental quantum primitives, namely the *[quantum Fourier transform][qft]* and *[quantum phase estimation][qpe]*. For a detailed explanation of the quantum aspects of Shor's algorithm, see the Phase estimation and factoring lesson in the [Fundamentals of Quantum Algorithms](https://learning.quantum-computing.ibm.com/course/fundamentals-of-quantum-algorithms). Steps 1 and 3 through 5 involve relatively inexpensive operations that can be easily carried out on classical computers.

Optionally, here's a detailed visual walkthrough of implementing [Shor's algorithm](https://www.youtube.com/watch?v=EdJ7RoWcU48&t=325s).

On quantum computers, Shor's algorithm can exhibit [polylogarithmic scaling][polyalg] as favorable as $O((log~n)^2 (log~log~n))$ in terms of the modulus $n$, or [polynomial scaling][poly] in terms of the number of bits needed to represent $n$. This is a [super-polynomial speedup][superpoly] compared to the classical [GNFS algorithm][gnfs].

[Recent resource estimates][39] indicate that based on certain assumptions made regarding the hardware configuration, a few tens of thousands to several million [qubits][qubit] will be necessary to break 2048-bit RSA using Shor's algorithm. It is not inconceivable that quantum computers with several tens of thousands of qubits will become available over the next several years, making the lower end of the resource estimate accessible.

[gnfs]: #definition-tooltip "General Number Field Sieve - An efficient classical algorithm for factoring large integers."
[period]: #definition-tooltip "The interval at which a function repeats itself."
[poly]: #definition-tooltip "Scales relative to a power, for example n^3. Grows quicker than linear scaling, but slower than exponential."
[polyalg]: #definition-tooltip "An algorithm which scales similar to logarithmic scaling, i.e. efficiently compared to the size of data. A binary search is a simple example."
[qft]: #definition-tooltip "A quantum algorithm that can be used to find periodic patterns. Named after the fourier transform in classical physics which can split a signal into a set of sine waves."
[qpe]: #definition-tooltip "A quantum algorithm used to estimate phase in qubits, named after phase estimation in classical physics."
[qubit]: #definition-tooltip "A quantum bit, the equivalent of a bit in classical computing. A qubit behaves very differently since it can be in superposition, (imagine a combination of states), but once measured the superposition collapses, and the result will be 0 or 1."
[superpoly]: #definition-tooltip "growing faster than a polynomial, for example an exponential function."
[39]: #definition-tooltip "Reference: C.Gidney and M. Ekerå,“How to factor 2048 bit RSA integers in 8 hours using 20 million noisy qubits,”Quantum, vol. 5, p. 433, Apr. 2021, [link](https://quantum-journal.org/papers/q-2021-04-15-433/)"
[76]: #definition-tooltip "Reference: “Shor’s algorithm,”Wikipedia. Jul. 2023. Accessed: Jul. 11, 2023. [link](https://en.wikipedia.org/w/index.php?title=Shor%27s_algorithm&oldid=1164409434)"

## Diffie-Hellman key exchange and the Digital Signature Algorithm

In the previous section, we discussed the RSA cryptosystem, whose security is based on the computational hardness of prime factorization. Here, we will discuss two popular asymmetric key cryptographic protocols, [*Diffie-Hellman* key exchange][62] (DH) and the [*Digital Signature Algorithm*][28] (DSA), both of which are based on a different mathematical problem, namely the discrete logarithm problem (DLP).

[62]: #definition-tooltip "Reference: Diffie–Hellman key exchange,Wikipedia. Jul. 2023. Accessed: Jul. 10,2023. [link](https://en.wikipedia.org/w/index.php?title=Diffie%E2%80%93Hellman_key_exchange&oldid=1163912552)"
[28]: #definition-tooltip "Reference: Digital Signature Algorithm,Wikipedia. Apr. 2023. Accessed:Jun. 20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Digital_Signature_Algorithm&oldid=1148950522)"

### The discrete logarithm problem

In the following equation we need to find $a$ given only $e$,$M$,$c$

$a^e$ $mod$ $M = c$

This is believed to be difficult with classic computers due to the use of modulo arithmetic, and therefore is a good mathematical basis for an encryption algorithm.

This is known as the [discrete logarithm problem][77] (DLP).

### Mathematical details of the discrete logarithm problem

The DLP is typically framed in the context of [cyclic groups][Cyclic] and is stated as follows.

Consider a [cyclic group][78] $G$ generated by a group element $g \in G$ and given an arbitrary element $h \in G$, find an integer $k$ such that $h = g^{k}$.

Here the integer $k \equiv log_{g}h$ is the discrete logarithm. The cyclic property of $G$ guarantees that for every $h$, a valid integer $k$ exists.

For cryptography, the DLP on the [multiplicative group][Multiplicative] of integers modulo a prime number $p$ denoted $(\mathbb{Z}_p)^{\times}$ turns out to be useful. The elements of $(\mathbb{Z}_p)^{\times}$ are congruence classes labeled by integers modulo $p$ that are coprime to $p$.

For example:

$(\mathbb{Z}_5)^{\times} = \{[1],[2],[3],[4]\}~\mathrm{and}~(\mathbb{Z}_7)^{\times} = \{[1],[2],[3],[4],[5],[6]\} $

The operation of multiplication $(\times)$ on these groups is simply ordinary integer multiplication followed by reduction modulo $p$ and exponentiation by an integer $k$ is just repeated multiplication $k$ times and reduction modulo $p$.

Let's illustrate an instance of the DLP on $(\mathbb{Z}_7)^{\times}$.

This multiplicative group has two generators ${[3],[5]}$ also known as primitive roots. We will use $[5]$ as the generator; that is, generate every element of $(\mathbb{Z}_7)^{\times}$ using successive integer powers of 5.

```python
#Generate elements of (Z_7)^{x} using generator [5].
g = 5
p = 7
print(f"Using generator {g}")
for k in range(3*p):
    print(f"{g}**{k} mod {p} ≡ {(g**k)%7}")
```

We see that in modulo 7 arithmetic, raising 5 to successive integer powers yields every element of $(\mathbb{Z}_7)^{\times}$ exactly once before the cycle repeats indefinitely with a period $p-1 =6$.

So the DLP on $(\mathbb{Z}_7)^{\times}$ with generator [5] is:

$ \mathrm{Given}~h \in (\mathbb{Z}_7)^{\times} \mathrm{, find~} k \mathrm{~such~that~} 5^{k} \equiv h~(mod~7)$.

From the above Python cell output we see that:

$h = 2 \implies k=4 \mathrm{~or~}  4 = log_5(2) (mod~7)$

$h = 6 \implies k=3 \mathrm{~or~}  3 = log_5(6) (mod~7)$


In ordinary real number arithmetic, exponentiation is a [monotonic][Monotonic], function and finding the logarithm of any number to any base is computationally easy. In contrast, as is apparent from the simple $(\mathbb{Z}_7)^{\times}$ example above, modular exponentiation is non-monotonic, and even though it is periodic with period $p-1$, it is otherwise highly nontrivial. So computing its inverse, the discrete logarithm turns out to be inefficient for large $p$ on classical computers.

This observation underpins both Diffie-Hellman (DH) key exchange and the Digital Signature Algorithm (DSA), which are discussed in the next section.

The DLP can be extended to cyclic subgroups as follows:

-   Consider $(\mathbb{Z}_p)^{\times}$ defined above and an element $g \in (\mathbb{Z}_p)^{\times}$ of prime order $r$ i.e., $g^r \equiv 1 (~mod~p)$.
-   The set of integer powers of $g$: $\{g^k~(mod~p) | 1 \le k \le r\} = \langle g \rangle$ is a cyclic subgroup of $(\mathbb{Z}_p)^{\times}$ with group order $r$.
-   A DLP can be specified on $\langle g \rangle$ by choosing a $h \in \\langle g \rangle$ and asking for $1 \le a \le r$ such that $ g^a~(mod~p) = h$

[Monotonic]: #definition-tooltip "a behavior or relationship that consistently moves in a single direction without reversals; for example in a monotonically increasing function, values consistently increase."
[Cyclic]: #definition-tooltip "Group where all elements are generated by repeatedly applying a single element, known as a generator. This process creates a cycle of elements that closes upon itself."
[Multiplicative]: #definition-tooltip "The set of integers modulo a prime number, where the group operation is modular multiplication. Elements in the multiplicative group are chosen in such a way that they are coprime to the modulus, ensuring that each element has a unique multiplicative inverse within the group."
[77]: #definition-tooltip "Reference: “Discrete logarithm,”Wikipedia. Jun. 2023. Accessed: Jul. 11, 2023. [link](https://en.wikipedia.org/w/index.php?title=Discrete_logarithm&oldid=1161792952)"
[78]: #definition-tooltip "Reference: “Cyclic group,”Wikipedia. Apr. 2023. Accessed: Jul. 11, 2023. [link](https://en.wikipedia.org/w/index.php?title=Cyclic_group&oldid=1150017831)"

### Diffie-Hellman key exchange

In 1976, [Whitfield Diffie and Martin Hellman proposed a key exchange protocol][79] to enable the creation of a shared secret key over insecure communication channels. The secret key could then be used by parties sharing it for symmetric encryption. The algorithm relies on the DLP.

![Figure 1. Diffie-Hellman key exchange](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/key_exchange_dh.avif)

*Figure 1. Diffie-Hellman key exchange*


<details>

<summary>

Mathematical details of Diffie-Hellman key exchange

</summary>
With Alice and Bob being the two parties communicating, the protocol works as follows:

- First, Alice and Bob agree on a large prime number $p$ and a primitive root or generator $a$.
- Then, Alice chooses a secret exponent $k_A$ randomly with $1 \le k_A \le p-2$ and calculates $h_A = a^{k_A}~(mod~p)$. $k_A, h_A$ are Alice's  private and public keys respectively.
- Next, Bob chooses a secret exponent $k_B$ randomly with  $1 \le k_B \le p-2$ and calculates $h_B = a^{k_B}~(mod~p)$. $k_B, h_B$ are Bob's  private and public keys respectively.
- Then, Alice sends Bob $h_A$ and Bob sends Alice $h_B$ over a reliable but not necessarily secure channel.
- Then, Alice then uses the $h_B$ she received to compute the shared secret key $ \kappa = h_B^{k_A}~(mod~p)$.
- Finally, Bob meanwhile uses the $h_A$ he received to compute the shared secret key $ \kappa = h_A^{k_B}~(mod~p)$.

With this protocol,

- Alice and Bob are guaranteed to end up with the same secret key $\kappa$ because $h_B^{k_A}~(mod~p) = (a^{k_B})^{k_A}~(mod~p) = a^{k_A k_B}~(mod~p) = (a^{k_A})^{k_B}~(mod~p) = h_A^{k_B}~(mod~p) $.
- A third party that intercepts $h_A$ or $h_B$ cannot construct the secret key $\kappa$ because they do not have access to $k_B$ or $k_A$ respectively.
- Extracting $k_A$ or $k_B$ using the public information $a$, $p$, $h_A$ and $h_B$ is computationally hard as it is equivalent to solving the DLP on $(\mathbb{Z}_p)^{\times}$.

</details>

[79]: #definition-tooltip "Reference: W.Diffie and M. Hellman,“New directions in cryptography,”IEEE Transactions on Information Theory, vol. 22, no. 6, pp.644–654, Nov. 1976, [link][https://doi.org/10.1109/TIT.1976.1055638)"

#### Illustration of the Diffie-Hellman protocol in Python

Let's look at a simple example of the DH protocol in Python using small prime numbers:

> Note: In this section we will show the math calculations in detail as part of the Python code

**Step 1:** Alice and Bob agree on a prime $p$ and a primitive root $a$. Let's choose $p=11, a=7$.

In [ ]:
# Step 1: Choose a prime `p` and a primitive root `a`
p = 11
a = 7

print(f"prime: {p}")
print(f"primitive root: {a}")

**Steps 2, 3:** Alice chooses a secret exponent $k_A$ and calculates $h_A = a^{k_A}~(mod~p)$. Similarly, Bob chooses a secret exponent $k_B$ and calculates $h_B = a^{k_B}~(mod~p)$.

In [ ]:
k_A = 4  # Alice's private key
h_A = (a ** (k_A)) % p  # Alice's public key

print(f"Alice's private key is {k_A} and public key is {h_A}")

k_B = 8  # Bob's private key
h_B = (a ** (k_B)) % p  # Bob's public key

print(f"Bob's private key is {k_B} and public key is {h_B}")

**Step 4:** The two parties broadcast their public keys $h_A$ and $h_B$.

**Steps 5, 6:** Each party combines their private key with the other party's public key to create the shared secret key.

In [ ]:
secret_key_alice = h_B**k_A % p
secret_key_bob = h_A**k_B % p
assert secret_key_alice == secret_key_bob
print(f"The shared secret key is: {secret_key_bob}")

Alice and Bob can now use the shared secret key for symmetric encryption.

#### Security of Diffie-Hellman key exchange

As noted above, the security of DH is predicated on the computational difficulty of solving the DLP with large primes $p$. In typical applications, NIST recommends 2048- or 3072-bit prime integers for DH key exchange, which is considered sufficiently secure against attempts to solve the DLP using classical computers.

**Man-in-the-middle (MITM) attacks**: The fact that DH is an interactive scheme where the shared secret depends on combining one party's private key with the other party's public key makes it vulnerable to a so-called *Man-in-the-middle (MITM)* attack.

<details>

<summary>

Mathematical details of DH security and MITM attacks

</summary>

In this scenario, a third party — say, Eve — intercepts the public keys $h_A, h_B$ during transmission and substitutes her own public key $h_E$ for each of $h_A$ and $h_B$ before forwarding them along to Bob and Alice, respectively.

Then, instead of using $h_B$ to create her shared secret, Alice will use $h_E$ while thinking that she is using Bob's public key. Similarly, instead of using $h_A$ to create his shared secret, Bob will use $h_E$ while thinking that he is using Alice's public key.

Because $h_E$ was used to create Alice's (Bob's) shared secret, plain text encrypted by Alice (Bob) can be decrypted by Eve.

Thus, DH key exchange is typically used in conjunction with a digital signature algorithm to ensure that each party uses an authenticated public key for creating their shared secret.

</details>

### The Digital Signature Algorithm (DSA)

Even though generic cryptosystems like [RSA][66] provide digital signature functionality, in 1994 NIST adopted a specialized signature scheme based on modular exponentiation and the discrete logarithm problem as the federal standard for digital signatures. This scheme, which came to be known simply as the [*Digital Signature Algorithm*][28] (DSA), involves four distinct phases:


[66]: #definition-tooltip "Reference: RSA(cryptosystem),Wikipedia. Jul. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=RSA_(cryptosystem)&oldid=1163962255)"
[28]: #definition-tooltip "Reference: Digital Signature Algorithm,Wikipedia. Apr. 2023. Accessed:Jun. 20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Digital_Signature_Algorithm&oldid=1148950522)"

1. **Key generation**:

   DSA keys are generated from:
     - 2 primes that meet certain rules
       -   $p$ - typically 256 bits (we'll call this length $N$)
       -   $q$ - typically 3072 bits (we'll call this length $L$)
     - A cryptographic hash function that will convert from strings of length $L$ to $N$
     - An additional Parameters $g$ (see details below)

   From this we choose a random number $x$ as private key, and are able to calculate a public key $y$

   <details>

   <summary>

   Mathematical details of key generation

   </summary>

   - **Parameter generation:** Mathematically, DSA involves a cyclic subgroup of $(\mathbb{Z}_p)^{\times}$ generated by an element $g$ of prime order q < p. The first step in the DSA is therefore to select two primes p, q to set up the necessary mathematical structures.

     - Choose an $N$-bit prime $q$.
     - Choose an $L$-bit prime $p$ such that $p-1$ is a multiple of $q$. The choice of $p$ specifies the group $(\mathbb{Z}_p)^{\times}$
     - Pick an integer $1 < h < p-1$ at random and compute $g = h^{(p-1)/q}~mod~p$. If $g=1$ which happens rarely, try another h.
     - Verify that $g^q~mod~p = 1$. g is thus a generator of a cyclic subgroup $\langle g \rangle$ of $(\mathbb{Z}_p)^{\times}$ of prime order q.

     The parameters $(q, p, g)$ specify an instance of the algorithm and are public information. Typically, $ N \sim 256$ and $L \sim 3072$ in realistic applications.

     The protocol also requires a cryptographic hash function $H : \{0,1\}^L \rightarrow \{0, 1\}^N$ that maps binary $L$-bit strings to $N$-bit strings, for example, SHA-256.

   - **Key-pair generation:** The signer needs to generate a key pair on their end.

     - Choose a random secret integer $ x \in \{1,2...q-1\}$. $x$ is the private key.
     - Generate $ y = g^{x}~mod~p$. $y$ is the signer's public key.

   </details>

2. **Key distribution:**

   The following information is shared with anyone who wishes to validate a signature
   - The parameters used $(p,q,g)$ which describe the algorithm
   - The hash function $H$
   - the public key $y$

1. **Signing:**

   - The signer can now sign a message $m$. The resulting signature is $(r,s)$
   - The message and signature are both now sent to the recipient

   <details>

   <summary>

   Mathematical details of signing a message

   </summary>

   The signer signs a message $m$ as follows:

   - Choose a secret integer k at random from $\{1,2...q-1\}$
   - Compute $r = (g^k~mod~p)~mod~q$. In the rare instance when $r=0$, try another random k.
   - Compute $s = (k^{-1} (H(m) + xr))~mod~q$. In a rare instances if $s=0$, try another random k.
   - The signature is the tuple $(r, s)$.
   - The signer transmits the message $m$ as well as the signature $(r,s)$ to the verifier.

   Because both $r, s$ are integers, modulo $q$ the signature size is on the order of $N$-bits and not the longer $L$-bits.

   </details>

2. **Verification:**

   The recipient now wishes to verify the authenticity of the sender.
   They have access to the public information $(q, p, g, H, y, (r,s), m)$
   They can execute a calculation which proves that the sender had access to the private key $x$

   and seeks to ascertain that the signer is someone with access to the private key $x$.

   <details>

   <summary>

   Mathematical details of DSA verification scheme

   </summary>

   - Verify that $0 \lt r \lt q$ and $0 \lt s \lt q$, i.e., $r, s$ are valid modulo~q integers.
   - Compute $w = s^{-1}~mod~q$.
   - Compute $u_1 = H(m)w~mod~q$.
   - Compute $u_2 = rw~mod~q$.
   - Compute $v = (g^{u_1}y^{u_2}~mod~p)~mod~q$.
   - The signature is verified if $v = r$.

   For legitimate signatures, the correctness of the DSA algorithm is easily demonstrated as follows:

   - On the signer's end: $s = (k^{-1} (H(m) + xr))~mod~q \implies k = ((H(m) + xr)s^{-1})~mod~q $
   - Considering the right-hand side of the latter equality, a verifier can compute $s^{-1}, H(m)$ because $s, q, m, H$ are public information.
   - Thus, the verifier computes $w = s^{-1}~mod~q$ and $u_1 = H(m)w~mod~q = H(m)s^{-1}~mod~q$.
   - The verifier also computes $u_2 = rw~mod~q = rs^{-1}~mod~q$ as $r$ is also public.
   - Note that $k = (u_1 + xu_2)~mod~q$.
   - However, a verifier does not have access to $x$ as it is the signer's private key, so $k$ itself cannot be directly computed.  - The scheme instead relies on the fact that even as the verifier cannot compute $k$, with a legitimate signer, the verifier should instead be able to re-compute $(g^k~mod~p)~mod~q = r$ with the help of the signer's public key $y = g^{x}~mod~p$.
   - So, the verifier computes $v = (g^{u_1}y^{u_2}~mod~p)~mod~q = (g^{u_1}g^{xu_2}~mod~p)mod~q = (g^{u_1+xu_2}~mod~p)mod~q = (g^k~mod~p)mod~q$.
   - The last equality follows because $g$ has order $q$ and establishes $v = r$ for valid signatures.

   </details>

### Illustration of the DSA in Python

In this example, we'll use small primes to illustrate the DSA process in a scenario where Alice sends a signed message to Bob. We'll use small integers in this example. A real-world scenario would employ much larger primes on the order of 2048-3072 bits.

> You can try rerunning this example with different values to see how the algorithm behaves, but ensure you start running from the first cell in this section.

We start by importing necessary libraries and choosing our parameters.

The integer parameters $p$, $q$, $g$ are public information, and satisfy the following rules:

- $p$, $q$ are both prime
- $(p-1) \mod\ q = 0$
- $g \ge 2$
- $g \le (p-2)$
- $g^{(p-1)/q} \mod\ p \neq 1$

In [ ]:
from random import randint

# parameter generation: select the primes q, p and generator g:
# EXPERIMENT with the values, they must meet certain rules
# this example code does not verify p,q are prime

q = 11
p = 23
g = 4

assert (p - 1) % q == 0
assert g >= 2
assert g <= (p - 2)
assert (pow(g, (p - 1) / q) % p) != 1

print(f"Public information is good: q={p}, p={q}, g={g}")

Next the signer, Alice, generates her private key.

The private key, k (`alice-private-key` in the python code) must satisfy:
 - $k \ge 2$
 - $k \le (q-1)$

In [ ]:
# Alice chooses an integer randomly from {2..q-1}
# EXPERIMENT with the values

alice_private_key = randint(2, q - 1)
# alice_private_key =

assert alice_private_key >= 2
assert alice_private_key <= (q - 1)

print(f"Alice's private key is {alice_private_key}")

Alice keeps her private key secret.

Next, Alice computes her public key using modular exponentiation. Inverting this step to recover the private key is an instance of the DLP, so difficult to compute on classical computers where large primes are used.

From the math explanation above, we know is this can be calculated using the formula:

$y = g^{x} \mod\ p$

In [ ]:
alice_public_key = pow(g, alice_private_key, p)
# Alternatively can use (g ** alice_private_key) % p

print(f"Alice's public key is {alice_public_key}")

As usual, Alice makes her public key available over a not-necessarily secret channel.

Now Alice can sign a message.

For the digital signature scheme, we need to generate a hash $H(m)$ of the message $m$ to be signed.

Let's assume Alice and Bob agree to use a particular hashing algorithm with hash length $N$ equal to the number of bits in $q$. In this simple example, we will bound the outputs of our mock hash function by $q$. The hash function here is trivial, simply generating a random integer.

In [ ]:
hash_dict = {}


def mock_hash_func(input_message):
    print(input_message)
    if input_message not in hash_dict:
        hash_dict[input_message] = randint(1, q)
    return hash_dict[input_message]


alice_message = "Inspection tomorrow!"
alice_hash = mock_hash_func(alice_message)  # In reality, you'd use a hash function
print(f"Alice's message hash is: {alice_hash}")

Next, Alice needs to generate a random per-message secret number $k$ as well as its multiplicative inverse modulo $q$ to compute the signature.

A simple brute-force algorithm can be used to compute the modular inverse. However it's better to use one of the built-in Python function `pow` as shown below

In [ ]:
# brute-force implementation to find modular inverse
def modular_inverse(k, q):
    for i in range(0, q):
        if (k * i) % q == 1:
            return i
    print(f"error! no inverse found! for {k},{q}")
    return 0


# Let's compare this algorithm with the standard python 'pow' function

n1 = modular_inverse(3, 7)
n2 = modular_inverse(4, 11)
n3 = modular_inverse(7, 5)
m1 = pow(3, -1, 7)
m2 = pow(4, -1, 11)
m3 = pow(7, -1, 5)

assert n1 == m1
assert n2 == m2
# assert(n3==m3)

print(f"modular_inverse(3,7) = {m1}")
print(f"modular_inverse(4,11) = {m2}")
print(f"modular_inverse(7,5) = {m3}")


# Some numbers don't have modular inverses - our function throws an error
n4 = modular_inverse(2, 6)

# The python library will throw an exception, which must be caught
if math.gcd(2, 6) == 1:
    m4 = pow(2, -1, 6)
else:
    print("Exception from pow() - no modular inverse found!")

Alice can now compute the signature.

- $r = (g^{k} \mod p) \mod\ q$
- $s = (k^{-1} (H(m) + xr)) \mod q$

Note that there are some particular conditions that will require us to choose a different value for $k$ in the case where either $r$ or $s$ compute to $0$. In this case we will generate a new value and repeat.

In [ ]:
# Start an infinite loop; we will 'break' out of it once a valid signature is found.
while True:
    k = randint(1, q - 1)  # Should be different for every message
    print("Using random k =", k)

    r = pow(g, k, p) % q
    # If r is 0, the value is invalid. Try again with a new k.
    if r == 0:
        print(f"{k} is not a good random value to use to calculate r. Trying another k")
        continue

    s = (pow(k, -1, q) * (alice_hash + alice_private_key * r)) % q
    # If s is 0, the value is also invalid. Try again with a new k.
    if s == 0:
        print(f"{k} is not a good random value to use to calculate s. Trying another k")
        continue

    # If we've reached this point, both r and s are valid. Break the loop.
    signature = (r, s)
    print(f"Alice's signature is : {(r,s)}")
    break

# After the loop, the valid r and s values can be used here.
# print(f"Generated Signature -> r: {r}, s: {s}")

Alice broadcasts the message and her signature to the receiver or verifier, Bob.

Bob has previously intercepted Alice's public key and can now verify the signature to authenticate her message.

To do so, he performs some checks, and then re-generates a hash of Alice's broadcast message and computes the auxiliary quantities $w, u_1, u_2$ and $v$.
- $0 < r < q$
- $0 < s < q$
- $w = s^{-1} \mod\ q$
- $u_{1} = H(m) . w \mod\ q$
- $u_{2} = r . w \mod\ q$
- $v = (g^{u1}y^{u2} \mod\ p) \mod\ q$


Finally, Bob checks if $v$ equals $r$. If they are equal, then the signature is verified.

In [ ]:
# Bob re-generates message hash using Alice's broadcast message
bob_hash = mock_hash_func(alice_message)

# Bob computes auxiliary quantities w (using modular inverse), u1, u2 and v
w = (pow(s, -1, q)) % q
u1 = (bob_hash * w) % q
u2 = (r * w) % q
v = ((g**u1 * alice_public_key**u2) % p) % q

if v == r:
    print("Signature is valid!")
else:
    print("Signature is invalid!")

### Security of the DSA

With classical computing, the security of the DSA can be influenced by several factors:

1. **Key size**: As always, key size provides basic protection against brute force attacks. Modern applications that use DSA use key sizes of at least 2048 bits.

2. **Quality of $k$**: In DSA, each signature needs a unique, random, and secret $k$ value (see above). The uniqueness, entropy, and secrecy of $k$ are critical, and weakness in any of these aspects could lead to the private key $x$ being compromised. The random number generators used for generating $k$ need to be secure themselves.

3. **Hash function strength**: Since DSA uses a hash function as part of the signature generation and verification process, a weak hash function could compromise the security of the digital signature. For example, the use of SHA-1 with DSA is deprecated and SHA-2 or higher is recommended.

In addition to these, robust DSA implementation should also guard against other types of attacks that target asymmetric key cryptography as previously outlined.

### Authenticated key exchange with Diffie-Hellman and DSA

Modern network security protocols, such as [Transport Layer Security (TLS)][127] and many others, involve combining the functionalities of key exchange and authentication. In the context of Diffie-Hellman, authentication is needed to guard against MITM attacks. In the following code cells, we illustrate an example where Alice and Bob perform an authenticated key exchange by combining the Diffie-Hellman and DSA protocols. For this, we will use the `cryptography` Python library.

![Figure 2. Authenticated key exchange with Diffie-Hellman and DSA](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/auth_key_exchange.avif)

*Figure 2. Authenticated key exchange with Diffie-Hellman and DSA*

[127]: #definition-tooltip "Reference: “Transport Layer Security,”Wikipedia. May 2023. Accessed:May 18, 2023. [link](https://en.wikipedia.org/w/index.php?title=Transport_Layer_Security&oldid=1155601531)"

First, Alice and Bob agree on a set of DH parameters and generate a set of DH public-private key pairs.

In [ ]:
# import necessary library modules
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh, dsa

# Generate DH parameters
parameters = dh.generate_parameters(generator=2, key_size=2048)

# Generate Alice's DH private key and public key
alice_dh_private_key = parameters.generate_private_key()
alice_dh_public_key = alice_dh_private_key.public_key()

# Generate Bob's DH private key and public key
bob_dh_private_key = parameters.generate_private_key()
bob_dh_public_key = bob_dh_private_key.public_key()

print("Public and private keys generated for Bob and Alice")

The DH public keys are broadcast. Next, both Alice and Bob each generate a separate key pair for use with DSA. These keys will in turn be used to sign the DH public keys to be exchanged.

In [ ]:
# Generate DSA keys for Alice and Bob
alice_dsa_private_key = dsa.generate_private_key(key_size=2048)
alice_dsa_public_key = alice_dsa_private_key.public_key()
bob_dsa_private_key = dsa.generate_private_key(key_size=2048)
bob_dsa_public_key = bob_dsa_private_key.public_key()

print("Additional key pair generated for signing")

Alice signs her DH public key using her DSA private key.

In [ ]:
# Alice signs
alice_public_bytes = alice_dh_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
alice_signature = alice_dsa_private_key.sign(alice_public_bytes, hashes.SHA256())

print("Alice signed public key")

Similarly, Bob signs his DH public key using his DSA private key.

In [ ]:
bob_public_bytes = bob_dh_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
bob_signature = bob_dsa_private_key.sign(bob_public_bytes, hashes.SHA256())

print("Bob signed public key")

The DH public keys and corresponding signatures are now broadcast by both Alice and Bob. Having received their counterparty's public key and signature, each party then verifies the signature is valid. This way, a MITM attack can be prevented, as Alice, for instance, knows that Bob's DH public key was indeed signed by Bob and vice versa.

In [ ]:
# Alice and Bob verify each other's DH public keys using DSA public keys
# An InvalidSignature exception will occur if they are not valid
alice_dsa_public_key.verify(alice_signature, alice_public_bytes, hashes.SHA256())
bob_dsa_public_key.verify(bob_signature, bob_public_bytes, hashes.SHA256())

print("Signatures are valid")

Following signature verification, Alice and Bob generate the shared secret, which completes the key exchange.

In [ ]:
# Perform key exchange
alice_shared_key = alice_dh_private_key.exchange(bob_dh_public_key)
bob_shared_key = bob_dh_private_key.exchange(alice_dh_public_key)

print("Shared secrets generated")

Optionally, for additional security, Alice and Bob can use a specialized key derivation function such as *HKDF* to generate a more secure symmetric key from their shared secret using *key stretching* techniques.

In [ ]:
# Derive a shared symmetric key using key-stretching
def derive_key(shared_key):
    return HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=None,
    ).derive(shared_key)


alice_symmetric_key = derive_key(alice_shared_key)
bob_symmetric_key = derive_key(bob_shared_key)

assert alice_symmetric_key == bob_symmetric_key
print("Keys checked to be the same")

## Breaking Diffie-Hellman and DSA

Both the [Diffie-Hellman][62] (DH) and [DSA][28] protocols involve the generation of public keys of the form $ y = g^{x}~mod~p$, where the private key $x$ is the discrete logarithm.

An attacker who can solve an instance of the DLP can expose the private key of one of the two parties and, by combining it with the public key of the second party, get access to the shared secret. In the case of DSA, an attacker who can solve the DLP can recover the signer's private key, voiding the basic premise of a digital signature.

On classical computing systems, the DLP is believed not to have a polynomial-time solution. But as shown by Peter Shor in his [original 1994 paper][6], Shor's algorithm also solves the DLP in polynomial-time on quantum computers.

[62]: #definition-tooltip "Reference: Diffie–Hellman key exchange,Wikipedia. Jul. 2023. Accessed: Jul. 10,2023. [link](https://en.wikipedia.org/w/index.php?title=Diffie%E2%80%93Hellman_key_exchange&oldid=1163912552)"
[28]: #definition-tooltip "Reference: Digital Signature Algorithm,Wikipedia. Apr. 2023. Accessed:Jun. 20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Digital_Signature_Algorithm&oldid=1148950522)"
[6]: #definition-tooltip "Reference: P.W. Shor,“Algorithms for quantum computation: Discrete logarithms and factoring,”in Proceedings 35th Annual Symposium on Foundations of Computer Science, Nov. 1994, pp. 124–134. [link](https://doi.org/10.1109/SFCS.1994.365700)."

### Shor's algorithm and the discrete logarithm problem

Shor's algorithm is able to solve the discrete logarithm problem in polynomial time. It primarily achieves its efficiency by using quantum algorithms which can find the period of a function which depends on the input to the problem - which is then combined with more conventional operations.

<details>

<summary>

Mathematical details of Shor's algorithm

</summary>

Shor's algorithm provides an efficient solution to the DLP by mapping it to a more general problem in group theory known as the [hidden subgroup problem][80] (HSP).

In the HSP, one is given an algebraic group $G$ and a function $f: G \rightarrow X$  from $G$ to some set $X$ such that $f$ is constant on each coset of some subgroup $H$ of $G$ (and distinct on different cosets). Then, the task is to determine $H$. It is known that quantum computers can solve the HSP on finite Abelian groups in time polynomial in $log~|G|$ where $|G|$ is the group order.

In the case of the integer DLP discussed in this lesson, the mapping to the HSP is as follows:

- Let $p$ be a prime and consider the DLP given by $ y = g^r~mod~p$ where $g$ is a generator of $(\mathbb{Z}_p)^{\times}$. Since $g^{p-1} \equiv 1~mod~p$, $g$ has order $p-1$.
- Choose $G = \mathbb{Z}_{p-1} \times \mathbb{Z}_{p-1}$ where $\mathbb{Z}_{p-1}$ is the group of integers modulo $(p-1)$.
- Choose $f : G \rightarrow (\mathbb{Z}_p)^{\times}$ given as $f(a, b) = g^a y^{-b}~mod~p \equiv g^{a-rb}~mod~p$ .
- The kernel of $f$ is then the subgroup $H_0 = \langle (r, 1) \rangle = \{(a,b) \in G | f(a,b) \equiv 1~mod~p\} = \{ (a, b) \in G | a - rb \equiv 0~mod~(p-1)\}$.
- $f$ is constant on the cosets $(\delta, 0) + H_0$ and "hides" the subgroup $H_0$ setting up an HSP.

Given the above, Shor's quantum algorithm for the integer DLP uses a quantum oracle to evaluate the function $f$ on a state representing a uniform superposition over $G$, and then uses the [quantum Fourier transform][qft] and measurements with [phase estimation][qpe] to produce quantum states that allow for the identification of the generator $(r, 1)$ of $H_0$. From this, $r$, the discrete logarithm of interest, can be extracted.

</details>

[Shor's original paper][6] has a detailed description of the algorithm.

[qft]: #definition-tooltip "A quantum algorithm that can be used to find periodic patterns. Named after the fourier transform in classical physics which can split a signal into a set of sine waves."
[qpe]: #definition-tooltip "A quantum algorithm used to estimate phase in qubits, named after phase estimation in classical physics."
[6]: #definition-tooltip "Reference: P.W. Shor,“Algorithms for quantum computation: Discrete logarithms and factoring,”in Proceedings 35th Annual Symposium on Foundations of Computer Science, Nov. 1994, pp. 124–134. [link](https://doi.org/10.1109/SFCS.1994.365700)."
[80]: #definition-tooltip "Reference: “Hidden subgroup problem,”Wikipedia. May 2023. Accessed: Jul. 12, 2023. [link](https://en.wikipedia.org/w/index.php?title=Hidden_subgroup_problem)"

## Elliptic curve cryptography

[*Elliptic curve cryptography (ECC)*][33], based on the mathematics of [*elliptic curves*][82], offers a powerful approach to asymmetric key cryptography. ECC is known to provide a similar level of security as methods such as RSA but with shorter keys, making it more efficient and particularly well suited to systems with limited resources, such as embedded systems and mobile devices, where the [storage and computational savings of smaller key sizes are highly desirable][65].

[33]: #definition-tooltip "Reference: Elliptic-curve cryptography,Wikipedia. Jun. 2023. Accessed: Jun. 20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic-curve_cryptography&oldid=1160709993)"
[65]: #definition-tooltip "Reference: C.A. Lara-Nino, A. Diaz-Perez, and M. Morales-Sandoval,“Elliptic Curve Lightweight Cryptography: A Survey,”IEEE Access, vol.6, pp. 72514–72550, 2018, [link](https://doi.org/10.1109/ACCESS.2018.2881444)"
[82]: #definition-tooltip "Reference: “Elliptic curve,”Wikipedia. Jul. 2023. Accessed: Jul. 12, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic_curve&oldid=1164567664)"

### Basic principles of elliptic curve cryptography

An elliptic curve is typically of the form $y^2 = x^3 + ax + b$ with a few interesting properties.
* It is horizontally symmetric. So for any point $(x,y)$ on the curve, it's reflection $(x,-y)$ is also on the curve
* Any non-vertical strain line will only intersect the curve at a maximum of three points

![Figure 1. Operations of addition and point doubling on an elliptic curve. Facet 1 defines <b>P + Q = -R</b>. Facet 2 illustrates point doubling <b>2Q = -P</b>. Facet 3 defines the additive inverse of a point as its reflection about the x-axis i.e, <b>P = -Q</b> ](/learning/images/courses/quantum-safe-cryptography/asymmetric-key-cryptography/ECCfig.avif)

*Figure 1. Operations of addition and point doubling on an elliptic curve. Facet 1 defines <b>P + Q = -R</b>. Facet 2 illustrates point doubling <b>2Q = -P</b>. Facet 3 defines the additive inverse of a point as its reflection about the x-axis i.e, <b>P = -Q</b>*

Elliptic curve cryptography makes use of applying a series of arithmetic operations to points on the curve. Each effectively navigates to a new point on the curve. This is a simple process to follow in one direction, and with shorter key sizes is more efficient than other algorithms like RSA, but trying to reverse this is very difficult, hence its application to cryptography.

<details>

<summary>

Mathematical principles of elliptic curve cryptography

</summary>

An [elliptic curve][82] over an algebraic field $K$ is defined by a mathematical equation, typically in the form $y^2 = x^3 + ax + b$ with the coefficients $a, b \in K$ and describes points $(x, y) \in K \otimes K$ with the additional requirement that the curve have no kinks or self-intersections.


The properties of elliptic curves allow for operations of "addition" and "scalar multiplication" to be defined involving points on the curve.


**Addition**: If $P$ and $Q$ are two points on an elliptic curve, then $P + Q$ describes a unique third point identified as follows: Draw the line that intersects $P$ and $Q$ and find the third point, $R$, at which the line intersects the curve again. We then define $P + Q =  −R$, the point opposite $R$ reflected through the $x$-axis (see figure below) . When the line through $P, Q$ does not intersect the curve at any finite $(x, y)$, we say it intersects the curve at the point at infinity $\mathbf{O}$.


Elliptic curve addition satisfies algebraic **group** properties with the point at infinity as the additive identity.


**Doubling and scalar multiplication:** The operation of point doubling which corresponds to scalar multiplication by $2$ is obtained by setting $P = Q$ in the addition operation and graphically involves the tangent line at $P$. General scalar multiplication by an integer $n$ defined as $nP =  P + P + ...~n$ times is obtained by repeated doubling and addition.

</details>

[82]: #definition-tooltip "Reference: “Elliptic curve,”Wikipedia. Jul. 2023. Accessed: Jul. 12, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic_curve&oldid=1164567664)"

### Elliptic curve discrete logarithm problem

The elliptic curve discrete logarithm problem (ECDLP) has many similarities to the discrete logarithm problem discussed previously, being based around the challenges of finding factors.

The operation of [scalar multiplication on elliptic curves][34] allows for the definition of the [elliptic curve discrete logarithm problem][33]:

Given points $P, Q$ on an elliptic curve such that $Q = nP$, determine $n$.

This problem is known to be intractable on classical computers for large $n$ and provides a basis for cryptographic security.

<details>

<summary>

Mathematical description of ECDLP

</summary>

Elliptic curve cryptography is primarily based on the ECDLP formulated on certain algebraic **finite fields**. In 1999, NIST recommended ten finite fields for use in ECC. These are:

 - Five **prime fields** $\mathbb {F} _{p}$ for primes $p$ of size $\{192, 224, 256, 384, 521\}$ bits.
 - Five **binary fields** $\mathbb {F} _{2^{n}}$ for $n \in \{163, 233, 283, 409, 571\}$.

With the above setup, an ECC-based asymmetric key cryptosystem in the case of **prime fields** may be specified as follows:

- Choose a $p$ from the NIST-recommended list of values to specify $\mathbb {F} _{p}$.
- Select parameters $a, b$ of the elliptic curve.
- Choose a **base point** $G$ which "generates" a cyclic subgroup on the curve with order $n$; that is, the smallest integer such that $nG = \mathbf{O}$.
- Calculate the **cofactor** $h = |E(\mathbb {F} _{p})|/n$ where $|E(\mathbb {F} _{p})|$ is the number of points on the curve. $h$ is typically a small integer.
- The **domain parameters** $(p, a, b, G, n, h)$ allow the specification of asymmetric keys in this way:

    - The private key is a randomly chosen integer $d$ with as many bits as in the prime $p$. It should be kept secret.
    - The public key is the result of "multiplying" the base point $G$ by the private key $d$. This is typically denoted as $Q = dG$.

Recovering $d$ is equivalent to solving the ECDLP, which is assumed to be intractable for large $d$. The ECDLP therefore forms the basis for key exchange and digital signature schemes in direct analogy with the Diffie-Hellman and DSA schemes defined over $(\mathbb{Z}_p)^{\times}$ discussed previously.

</details>

[33]: #definition-tooltip "Reference: Elliptic-curve cryptography,Wikipedia. Jun. 2023. Accessed: Jun. 20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic-curve_cryptography&oldid=1160709993)"
[34]: #definition-tooltip "Reference: “Elliptic curve point multiplication,”Wikipedia. Mar. 2023. Accessed: Jun.20, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic_curve_point_multiplication&oldid=1145917191)"

### Key exchange with ECC

One of the primary uses of ECC is in the key exchange protocol known as [elliptic curve Diffie-Hellman][63] (ECDH). In ECDH, each party generates a private-public key pair and then exchanges public keys. Each party then uses their own private key and the other party's public key to compute a shared secret, which can be used as the key for symmetric encryption.

While it's relatively easy to perform point addition on an elliptic curve and derive a public key from a private key, it's computationally infeasible to reverse the process and derive a private key from a public key. This "one-way" behavior provides the security of the ECDH key exchange.

Here, we will illustrate an example of how one might perform an ECDH key exchange using Python and the library `cryptography`.

[63]: #definition-tooltip "Reference: Elliptic-curve Diffie–Hellman,Wikipedia.Jun. 2023. Accessed: Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic-curve_Diffie%E2%80%93Hellman&oldid=1160050860)"

In [ ]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes

# Each party generates a private key
private_key1 = ec.generate_private_key(ec.SECP384R1())
private_key2 = ec.generate_private_key(ec.SECP384R1())

# They exchange public keys
public_key1 = private_key1.public_key()
public_key2 = private_key2.public_key()

# Each party uses their own private key and the other party's public key
# to derive the shared secret
shared_key1 = private_key1.exchange(ec.ECDH(), public_key2)
shared_key2 = private_key2.exchange(ec.ECDH(), public_key1)

# The shared secrets are the same
assert shared_key1 == shared_key2
print("Keys checked to be the same")

### Digital signatures with ECC

ECC can also be used to generate digital signatures using the [Elliptic Curve Digital Signature Algorithm][64] (ECDSA). In ECDSA, the signer creates a signature using their private key, and others can verify the signature using the signer's public key. Just like with ECDH, the security of ECDSA relies on the ECDLP. It's computationally infeasible for someone to forge a signature without access to the signer's private key.

The following is an example of a simple sign-and-verify transaction using ECDSA with Python and `cryptography`.

[64]: #definition-tooltip "Reference: Elliptic Curve Digital Signature Algorithm,Wikipedia. Jun. 2023. Accessed:Jul. 10, 2023. [link](https://en.wikipedia.org/w/index.php?title=Elliptic_Curve_Digital_Signature_Algorithm&oldid=1162097661)"

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec

# Generate a private key for use in the signature
private_key = ec.generate_private_key(ec.SECP384R1())

message = b"A message to be signed"

# Sign the message
signature = private_key.sign(message, ec.ECDSA(hashes.SHA256()))

# Anyone can verify the signature with the public key
public_key = private_key.public_key()


def check_ecdsa_signature(public_key, signature, message):
    try:
        public_key.verify(signature, message, ec.ECDSA(hashes.SHA256()))
        return True
    except InvalidSignature:
        return False


if check_ecdsa_signature(public_key, signature, message):
    print("The signature is valid.")
else:
    print("The signature is invalid.")

In the above code, if one modifies the message after it has been signed, the verification will fail, providing a guarantee of authenticity and integrity for the message.

### Breaking ECDH and ECDSA

In an analogous way to the integer discrete logarithm problem, the ECDLP turns out to be hard on classical computers but has an efficient solution on quantum computers once again via Shor's algorithm. We refer you to recent literature for a discussion related to [generalizing Shor's algorithm to the ECDLP case][41].

[41]: #definition-tooltip "Reference: H.T. Larasati and H. Kim,“Quantum Cryptanalysis Landscape of Shor’s Algorithm for Elliptic Curve Discrete Logarithm Problem,”in Information Security Applications, H. Kim, Ed., in Lecture Notes in Computer Science. Cham: Springer International Publishing, 2021, pp.91–104. [link](https://doi.org/10.1007/978-3-030-89432-0_8)"

## Summary

In this lesson, we started by looking at the main characteristics of asymmetric key cryptography (AKC) and discussed basic security considerations that underpin asymmetric cryptosystems. In particular, we introduced the two main applications of AKC — namely, key exchange and digital signatures — both of which are essential components of modern internet-based communication.

We then looked at the RSA cryptosystem, which since the 1970s, has proven to be of immense value for securing modern digital communications by enabling key exchange and digital signatures within a simple and versatile framework. The cryptographic security of RSA with respect to classical computing is based on the hardness of factoring large integers and requires key sizes in the range of 2048 bits to assure the integers used in practical applications are large enough to resist factorization.

We then looked at Diffie-Hellman (DH) key exchange and the Digital Signature Algorithm (DSA). The security of these schemes is predicated on the integer discrete logarithm (DLP) problem, the private key being computationally hard to extract given the public key, with no polynomial time solution on classical computers. Use of random and unique keys affords additional security against attacks. Both the integer finite field and elliptic curve variants of the DH and DSA protocols currently find widespread use across many modern digital communications protocols such as [*TLS*][127], [*SSH*][81], and so on.

Finally we looked at Elliptic curve cryptography. With its efficient key size and strong security properties, it currently represents an excellent choice for many cryptographic applications, from key exchange to digital signatures.

All of these algorithms have exposure to attack from quantum algorithms, since solutions can be developed to solve the mathematical problems which acted as a premise in their design. One such example is Shor's algorithm. Therefore they will need to be replaced by quantum-safe asymmetric cryptosystems which are more resilient to attack from quantum - as well as being as safe with classical algorithms. The mathematical problems they rely on can be solved efficiently by quantum computers, necessitating the development of quantum-safe asymmetric cryptosystems that can withstand quantum attacks while maintaining classical security.

A future lesson will look at quantum-safe cryptosystems and discuss the required approach to maintain cryptographic security.

[127]: #definition-tooltip "Reference: “Transport Layer Security,”Wikipedia. May 2023. Accessed:May 18, 2023. [link](https://en.wikipedia.org/w/index.php?title=Transport_Layer_Security&oldid=1155601531)"
[81]: #definition-tooltip "Reference: “SecureShell,”Wikipedia. Jun. 2023. Accessed: Jul. 12, 2023. [link](https://en.wikipedia.org/w/index.php?title=Secure_Shell&oldid=1158549718)"